# CODICE PARZIALE - Estrazione di tutti i dati di una singola ricetta a partire dall'url base: https://ricette.giallozafferano.it/Nome_ricetta.html + salvataggio su file excel e sQlite

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import sqlite3

# Definisci le colonne che saranno usate sia per Excel che per SQLite
COLUMNS = [
    'URL', 'Titolo', 'Presentazione', 'Calorie_per_porzione', 'Difficolta',
    'Dosi_per', 'Preparazione', 'Cottura', 'Costo', 'Ingredienti', 'Istruzioni'
]

def scrape_recipe(url):
    """Estrae le informazioni di una ricetta da un URL dato."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    title = soup.find('h1', class_='gz-title-recipe')
    title = title.text.strip() if title else 'Non specificato'

    presentation_div = soup.find('div', class_='gz-content-recipe')
    presentation = presentation_div.find('p').text.strip() if presentation_div else 'Non specificato'

    calories_div = soup.find('div', class_='gz-text-calories-total')
    calories = calories_div.find('span').text.strip() if calories_div else '0'
    calories = int(re.search(r'\d+', calories).group()) if re.search(r'\d+', calories) else 0

    difficulty = servings = preparation_time = cooking_time = cost = 'Non specificato'

    for span in soup.find_all('span', class_='gz-name-featured-data'):
        if 'Difficoltà' in span.text:
            difficulty = span.find('strong').text.strip()
        elif 'Dosi per' in span.text:
            servings = span.find('strong').text.strip()
            servings = int(re.search(r'\d+', servings).group()) if re.search(r'\d+', servings) else 0
        elif 'Preparazione' in span.text:
            preparation_time = span.find('strong').text.strip()
        elif 'Cottura' in span.text:
            cooking_time = span.find('strong').text.strip()
        elif 'Costo' in span.text:
            cost = span.find('strong').text.strip()

    ingredients = []
    for section in soup.find_all('dl', class_='gz-list-ingredients'):
        section_title = section.find('dt', class_='gz-title-ingredients')
        if section_title:
            ingredients.append(f"{section_title.text.strip()}:")
        for ingredient in section.find_all('dd', class_='gz-ingredient'):
            ingredients.append(' '.join(ingredient.text.split()))

    preparation = []
    for step in soup.find_all('div', class_='gz-content-recipe-step'):
        p_tag = step.find('p')
        if p_tag:
            preparation.append(p_tag.text.strip())
    return {
        'URL': url,
        'Titolo': title,
        'Presentazione': presentation,
        'Calorie_per_porzione': calories,
        'Difficolta': difficulty,
        'Dosi_per': servings,
        'Preparazione': preparation_time,
        'Cottura': cooking_time,
        'Costo': cost,
        'Ingredienti': '\n'.join(ingredients),
        'Istruzioni': '\n'.join(preparation)
    }

def save_to_excel(recipe_data, filename='ricette.xlsx'):
    """Salva o aggiorna i dati della ricetta in un file Excel."""
    df = pd.DataFrame([recipe_data], columns=COLUMNS)

    if os.path.exists(filename):
        existing_df = pd.read_excel(filename)
        if recipe_data['URL'] not in existing_df['URL'].values:
            updated_df = pd.concat([existing_df, df], ignore_index=True)
            updated_df.to_excel(filename, index=False, columns=COLUMNS)
            print(f"Nuova ricetta aggiunta al file esistente: {filename}")
        else:
            print(f"La ricetta con URL {recipe_data['URL']} esiste già nel file. Nessuna modifica apportata.")
    else:
        df.to_excel(filename, index=False, columns=COLUMNS)
        print(f"Nuovo file creato: {filename}")

def create_database():
    """Crea il database SQLite se non esiste."""
    conn = sqlite3.connect('ricette.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS ricette
                 (URL TEXT PRIMARY KEY,
                  Titolo TEXT,
                  Presentazione TEXT,
                  Calorie_per_porzione INTEGER,
                  Difficolta TEXT,
                  Dosi_per INTEGER,
                  Preparazione TEXT,
                  Cottura TEXT,
                  Costo TEXT,
                  Ingredienti TEXT,
                  Istruzioni TEXT)''')
    conn.commit()
    conn.close()

def save_to_database(recipe_data):
    """Salva o aggiorna i dati della ricetta nel database SQLite."""
    conn = sqlite3.connect('ricette.db')
    c = conn.cursor()

    c.execute('''INSERT OR REPLACE INTO ricette
                 (URL, Titolo, Presentazione, Calorie_per_porzione, Difficolta, Dosi_per,
                  Preparazione, Cottura, Costo, Ingredienti, Istruzioni)
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
              [recipe_data[col] for col in COLUMNS])

    conn.commit()
    conn.close()
    print(f"Ricetta salvata/aggiornata nel database: {recipe_data['Titolo']}")

def main():
    create_database()
    url = 'https://ricette.giallozafferano.it/Spezzatino-con-piselli.html'
    recipe_data = scrape_recipe(url)
    save_to_excel(recipe_data)
    save_to_database(recipe_data)

    for key, value in recipe_data.items():
        print(f"{key}:")
        print(f"{value}")
        print()

if __name__ == "__main__":
    main()

La ricetta con URL https://ricette.giallozafferano.it/Spezzatino-con-piselli.html esiste già nel file. Nessuna modifica apportata.
Ricetta salvata/aggiornata nel database: Spezzatino con piselli
URL:
https://ricette.giallozafferano.it/Spezzatino-con-piselli.html

Titolo:
Spezzatino con piselli

Presentazione:
Lo spezzatino di manzo è un classico della cucina casalinga poiché bastano pochi ingredienti per ottenere un secondo piatto ricco e succulento! Lo spezzatino con piselli è una delle tante varianti che si possono realizzare: la preparazione è alla portata di tutti, ma bisogna rispettare alcune indicazioni per un risultato ottimale. Innanzitutto la scelta del taglio di carne, il passaggio dell’infarinatura e della sigillatura, e infine il tempo di cottura… sono tutti elementi fondamentali per ottenere dei bocconcini di manzo morbidi e saporiti, immersi in una salsa cremosa e avvolgente. Seguite la ricetta e preparatevi a gustare uno spezzatino con piselli che si scioglie in bocca!



In [ ]:
import pandas as pd

# Leggi il file Excel
df = pd.read_excel('ricette.xlsx')

# Mostra le prime 10 righe
print(df.head(20))

                   Titolo                                      Presentazione  \
0  Spezzatino con piselli  Lo spezzatino di manzo è un classico della cuc...   

   Calorie per porzione Difficoltà  Dosi per Preparazione     Cottura  Costo  \
0                     0     Facile         4       15 min  1 h 40 min  Medio   

                                         Ingredienti  \
0  Manzo spalla 1,2 kg\nPisellini 400 g\nCipolle ...   

                                          Istruzioni  
0  Per preparare lo spezzatino con piselli per pr...  


# CODICE PARZIALE -Partendo dalla lista ingredienti, il seguente codice calcola calorie, proteine e grassi utilizzando le API dell'USDA FoodData Central. Fornisce sia i valori totali che parziali (per 100g)

In [ ]:
!pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 29.6 MB/s eta 0:00:00


In [ ]:
import requests
from rapidfuzz import process, fuzz

def get_nutritional_data(ingredient, api_key):
    search_url = f"https://api.nal.usda.gov/fdc/v1/foods/search?query={ingredient}&api_key={api_key}"
    response = requests.get(search_url)
    data = response.json()

    if data['foods']:
        # Cerca una corrispondenza esatta
        exact_match = next((food for food in data['foods'] if food['description'].lower() == ingredient.lower()), None)
        if exact_match:
            return exact_match, search_url, False, 100  # False indica che non è stata usata la ricerca fuzzy

        # Se non trova una corrispondenza esatta, usa la ricerca fuzzy
        best_match = process.extractOne(ingredient, [food['description'] for food in data['foods']], scorer=fuzz.WRatio)
        if best_match and best_match[1] >= 80:
            fuzzy_match = next(food for food in data['foods'] if food['description'] == best_match[0])
            similarity = best_match[1]
            return fuzzy_match, search_url, True, similarity  # True indica che è stata usata la ricerca fuzzy

    return None, search_url, False, 0

def calculate_nutrients(ingredients, api_key):
    total_calories = 0
    total_fat = 0
    total_protein = 0
    excluded_ingredients = []

    for ingredient, amount in ingredients.items():
        food_data, search_url, fuzzy_used, similarity = get_nutritional_data(ingredient, api_key)

        if food_data is None or similarity < 80:
            print(f"Ingrediente '{ingredient}' non trovato nel DB, procedere al calcolo manuale")
            excluded_ingredients.append(ingredient)
            continue

        nutrients = food_data['foodNutrients']
        calories_per_100g = next((item['value'] for item in nutrients if item['nutrientName'] == "Energy"), 0)
        fat = next((item['value'] for item in nutrients if item['nutrientName'] == "Total lipid (fat)"), 0)
        protein = next((item['value'] for item in nutrients if item['nutrientName'] == "Protein"), 0)

        # Calcolo delle calorie usando la formula CAL = D x P / 100
        calories = (calories_per_100g * amount) / 100

        print(f"\nInformazioni per {ingredient} (quantità: {amount}g):")
        if fuzzy_used:
            print(f"Nome approssimato trovato: {food_data['description']} (ricerca fuzzy, similarità: {similarity:.2f}%)")
        else:
            print(f"Nome trovato: {food_data['description']}")
        print(f"Densità calorica: {calories_per_100g} kcal/100g")
        print(f"Calorie totali: {calories:.2f} kcal")
        print(f"Grassi: {fat} g/100g")
        print(f"Proteine: {protein} g/100g")
        #print(f"URL ricerca: {search_url}")

        total_calories += calories
        total_fat += (fat / 100) * amount
        total_protein += (protein / 100) * amount

    result = {
        "calories": round(total_calories, 2),
        "fat": round(total_fat, 2),
        "protein": round(total_protein, 2)
    }

    if excluded_ingredients:
        print("\nATTENZIONE: I seguenti ingredienti sono stati esclusi dal calcolo:")
        for ingredient in excluded_ingredients:
            print(f"- {ingredient}")
        print("Procedere al calcolo manuale per questi ingredienti e aggiungere i risultati al totale.")

    return result

# Esempio di utilizzo
api_key = 'lRRUhBqakDfuwdjbdfEEck02bklmh6ml32q5ANxp'
ingredients = {
    "whole wheat biscuits": 180,
    "light butter": 80,
    "cream cheese, light": 500,
    "nonfat yogurt": 250,
    "fructose": 160,
    "gelatin": 10,
    "water": 30,
    "vanilla bean": 5,
    "cazzimbocchio": 100
}

nutrients = calculate_nutrients(ingredients, api_key)
print("\nNutrienti totali (esclusi gli ingredienti non trovati):")
print(nutrients)


Informazioni per whole wheat biscuits (quantità: 180g):
Nome approssimato trovato: Wheat flour, whole-grain, soft wheat (ricerca fuzzy, similarità: 85.50%)
Densità calorica: 1390.0 kcal/100g
Calorie totali: 2502.00 kcal
Grassi: 1.95 g/100g
Proteine: 9.61 g/100g

Informazioni per light butter (quantità: 80g):
Nome approssimato trovato: Butter-oil blend, light (ricerca fuzzy, similarità: 85.50%)
Densità calorica: 499 kcal/100g
Calorie totali: 399.20 kcal
Grassi: 55.1 g/100g
Proteine: 3.3 g/100g

Informazioni per cream cheese, light (quantità: 500g):
Nome trovato: Cream cheese, light
Densità calorica: 201 kcal/100g
Calorie totali: 1005.00 kcal
Grassi: 15.3 g/100g
Proteine: 7.85 g/100g

Informazioni per nonfat yogurt (quantità: 250g):
Nome trovato: NONFAT YOGURT
Densità calorica: 44.0 kcal/100g
Calorie totali: 110.00 kcal
Grassi: 0.0 g/100g
Proteine: 4.89 g/100g

Informazioni per fructose (quantità: 160g):
Nome approssimato trovato: Sweeteners, tabletop, fructose, liquid (ricerca fuzzy, s

# API call USDA

In [ ]:
import requests
import json

def get_usda_data(food_name, api_key):
    # URL dell'API USDA
    url = f"https://api.nal.usda.gov/fdc/v1/foods/search?api_key={api_key}&query={food_name}"

    # Effettua la richiesta GET
    response = requests.get(url)

    # Verifica se la richiesta ha avuto successo
    if response.status_code == 200:
        # Converte la risposta in JSON
        data = response.json()

        # Verifica se ci sono risultati
        if data['foods']:
            # Prende il primo risultato
            food = data['foods'][0]

            print(f"Nome: {food['description']}")
            print("Nutrienti:")

            # Stampa tutti i nutrienti
            for nutrient in food['foodNutrients']:
                print(f"  {nutrient['nutrientName']}: {nutrient['value']} {nutrient.get('unitName', '')}")
        else:
            print("Nessun risultato trovato.")
    else:
        print(f"Errore nella richiesta: {response.status_code}")

# Parametri
food_name = 'Eggs, Grade A, Large, egg yolk'
api_key = 'lRRUhBqakDfuwdjbdfEEck02bklmh6ml32q5ANxp'

# Chiamata alla funzione
get_usda_data(food_name, api_key)

# CODICE PARZIALE - Partendo da un dato file excel, il codice calcola calorie, proteine e grassi *utilizzando le API dell'USDA FoodData Central*, dettagliando sia il totale che i valori nutrizionali di ogni singolo ingrediente (per 100g)

In [ ]:
import pandas as pd
import requests
import re

# Funzione per ottenere i dati nutrizionali da USDA FoodData Central
def get_nutritional_data(ingredient, api_key):
    search_url = f"https://api.nal.usda.gov/fdc/v1/foods/search?query={ingredient}&api_key={api_key}"
    response = requests.get(search_url)
    data = response.json()
    # Prendi il primo risultato
    if data['foods']:
        return data['foods'][0]
    return None

# Funzione per analizzare gli ingredienti dal testo
def parse_ingredients(ingredients_text):
    ingredients = {}
    lines = ingredients_text.split('\n')
    for line in lines:
        match = re.search(r'([\w\s]+)\s(\d+)\s?g', line)
        if match:
            ingredient = match.group(1).strip()
            amount = int(match.group(2))
            ingredients[ingredient] = amount
    return ingredients

# Funzione per calcolare i nutrienti totali e per 100g di una ricetta
def calculate_nutrients(ingredients, api_key):
    total_calories = 0
    total_fat = 0
    total_protein = 0

    nutrients_per_100g = {}

    for ingredient, amount in ingredients.items():
        food_data = get_nutritional_data(ingredient, api_key)
        if food_data:
            nutrients = food_data['foodNutrients']
            calories = next((item['value'] for item in nutrients if item['nutrientName'] == "Energy"), 0)
            fat = next((item['value'] for item in nutrients if item['nutrientName'] == "Total lipid (fat)"), 0)
            protein = next((item['value'] for item in nutrients if item['nutrientName'] == "Protein"), 0)

            total_calories += (calories / 100) * amount
            total_fat += (fat / 100) * amount
            total_protein += (protein / 100) * amount

            nutrients_per_100g[ingredient] = {
                "calories_per_100g": calories,
                "fat_per_100g": fat,
                "protein_per_100g": protein
            }

    total_nutrients = {
        "total_calories": total_calories,
        "total_fat": total_fat,
        "total_protein": total_protein
    }

    return total_nutrients, nutrients_per_100g

# Carica il database da Excel
df = pd.read_excel('ricette_tradotte.xlsx')

# Funzione per calcolare i nutrienti di una ricetta specifica
def calculate_recipe_nutrients(recipe_title, api_key):
    recipe = df[df['Titolo'] == recipe_title].iloc[0]
    #ingredients_text = recipe['Ingredients']
    ingredients_text = recipe['Ingredients']
    ingredients = parse_ingredients(ingredients_text)
    print(ingredients)
    return calculate_nutrients(ingredients, api_key)

# Esempio di utilizzo
api_key = 'lRRUhBqakDfuwdjbdfEEck02bklmh6ml32q5ANxp'
recipe_title = "Spaghetti alla Carbonara"  # Sostituisci con il titolo della ricetta desiderata

total_nutrients, nutrients_per_100g = calculate_recipe_nutrients(recipe_title, api_key)
print(f"Nutrienti Totali per {recipe_title}:")
print(total_nutrients)
print("\nNutrienti per 100g di ogni ingrediente:")
for ingredient, nutrients in nutrients_per_100g.items():
    print(f"{ingredient}: {nutrients}")

FileNotFoundError: [Errno 2] No such file or directory: 'ricette_tradotte.xlsx'

# CODICE PARZIALE - normalizzazione della colonna 'Ingredients' e creazione di colonne dedicate cformattate in modo da recuperare gli ingredienti in modo semplice.

In [ ]:
import re

def parse_ingredients(text):
    # Rimuove la parte iniziale del testo che non contiene ingredienti
    ingredients_text = re.sub(r'^.*?Ingredients for.*?:', '', text, flags=re.DOTALL)

    # Divide il testo in righe
    lines = ingredients_text.split('\n')

    ingredients = {}

    for line in lines:
        # Usa una regex più flessibile per estrarre il nome dell'ingrediente e la quantità
        match = re.match(r'(.*?)\s*(\d+(?:\.\d+)?)\s*(g|to taste)?$', line.strip())
        if match:
            ingredient, quantity, unit = match.groups()

            # Pulisce e formatta il nome dell'ingrediente
            ingredient = ingredient.strip().lower()

            # Converte la quantità in float se possibile, altrimenti usa 0
            try:
                quantity = float(quantity)
            except ValueError:
                quantity = 0

            # Gestisce il caso "to taste"
            if unit == 'to taste':
                quantity = 'to taste'

            ingredients[ingredient] = quantity

    return ingredients

# Il testo di input
text = """
For about 12 pancakes:
00 flour 125 g
Whole milk 200 g
Butter 25 g
Sugar 15 g
Medium eggs 2
Baking powder for cakes 6 g
Salt 1 pinch
To grease the pan:
Butter to taste
to garnish:
Maple syrup to taste
More to taste
Raspberries to taste
"""

# Parsing degli ingredienti
result = parse_ingredients(text)

# Stampa il risultato
print(result)

{'00 flour': 125.0, 'whole milk': 200.0, 'butter': 25.0, 'sugar': 15.0, 'medium eggs': 2.0, 'baking powder for cakes': 6.0}


# EXTRA - Utilizzando *un file excel* come database di valori nutrizionali, il codice calcola calorie, proteine e grassi utilizzando, dettagliando sia il totale che i valori nutrizionali di ogni singolo ingrediente (per 100g)

In [ ]:
import pandas as pd
import re

# Carica i database
def load_databases():
    try:
        db_nutrizionale = pd.read_excel('dati_nutrizionali_modificati.xlsx')
        df_ricette = pd.read_excel('ricette.xlsx')
        return db_nutrizionale, df_ricette
    except FileNotFoundError as e:
        print(f"Errore nel caricamento dei file: {e}")
        return None, None

db_nutrizionale, df_ricette = load_databases()

# Funzione per ottenere i dati nutrizionali dal database personale
def get_nutritional_data(ingredient):
    matches = db_nutrizionale[db_nutrizionale['nome_alimento'].str.contains(ingredient, case=False, na=False)]
    if matches.empty:
        raise IndexError(f"Nessun dato trovato per l'ingrediente '{ingredient}'")
    return matches.iloc[0]

# Funzione per analizzare gli ingredienti dal testo
def parse_ingredients(ingredients_text):
    ingredients = {}
    lines = ingredients_text.split('\n')
    for line in lines:
        match = re.search(r'([\w\s]+)\s(\d+)\s?g', line)
        if match:
            ingredient = match.group(1).strip()
            amount = int(match.group(2))
            ingredients[ingredient] = amount
    return ingredients

# Funzione per calcolare i nutrienti totali e per 100g di una ricetta
def calculate_nutrients(ingredients):
    total_calories = 0
    total_protein = 0
    total_carbs = 0
    total_fat = 0

    nutrients_per_100g = {}

    for ingredient, amount in ingredients.items():
        try:
            food_data = get_nutritional_data(ingredient)
            print(f"Dati grezzi per '{ingredient}': {food_data.to_dict()}")

            calories = float(food_data['energia_kcal_100g'])
            protein = float(food_data['proteine'])
            carbs = float(food_data['carboidrati_g'])
            fat = float(food_data['colesterolo_mg'])

            total_calories += (calories / 100) * amount
            total_protein += (protein / 100) * amount
            total_carbs += (carbs / 100) * amount
            total_fat += (fat / 100) * amount

            nutrients_per_100g[ingredient] = {
                "calorie_per_100g": calories,
                "proteine_per_100g": protein,
                "carboidrati_per_100g": carbs,
                "grassi_per_100g": fat
            }
        except IndexError as e:
            print(f"Errore: {e}")
        except ValueError as e:
            print(f"Errore di conversione per l'ingrediente '{ingredient}': {e}")
        except KeyError as e:
            print(f"Colonna mancante nel database per l'ingrediente '{ingredient}': {e}")

    total_nutrients = {
        "calorie_totali": round(total_calories, 2),
        "proteine_totali": round(total_protein, 2),
        "carboidrati_totali": round(total_carbs, 2),
        "grassi_totali": round(total_fat, 2)
    }

    return total_nutrients, nutrients_per_100g

# Funzione per calcolare i nutrienti di una ricetta specifica
def calculate_recipe_nutrients(recipe_title):
    try:
        recipe = df_ricette[df_ricette['Titolo'] == recipe_title].iloc[0]
        ingredients_text = recipe['Ingredienti']
        ingredients = parse_ingredients(ingredients_text)
        print(f"Ingredienti analizzati: {ingredients}")
        return calculate_nutrients(ingredients)
    except IndexError:
        print(f"Ricetta '{recipe_title}' non trovata nel database.")
        return None, None

# Funzione principale
def main():
    if db_nutrizionale is None or df_ricette is None:
        print("Impossibile procedere a causa di errori nel caricamento dei database.")
        return

    recipe_title = input("Inserisci il titolo della ricetta: ")
    total_nutrients, nutrients_per_100g = calculate_recipe_nutrients(recipe_title)

    if total_nutrients and nutrients_per_100g:
        print(f"\nNutrienti Totali per {recipe_title}:")
        for nutrient, value in total_nutrients.items():
            print(f"{nutrient}: {value}")

        print("\nNutrienti per 100g di ogni ingrediente:")
        for ingredient, nutrients in nutrients_per_100g.items():
            print(f"{ingredient}:")
            for nutrient, value in nutrients.items():
                print(f"  {nutrient}: {value}")

if __name__ == "__main__":
    main()

Inserisci il titolo della ricetta: Spaghetti alla Carbonara
Ingredienti analizzati: {'Spaghetti': 320, 'Guanciale': 150, 'Pecorino romano': 50}
Dati grezzi per 'Spaghetti': {'id_alimento': 'PE0009', 'nome_alimento': 'Spaghetti fritti', 'categoria': 'Alimenti Etnici', 'parte_edibile': 1.0, 'porzione': '300 g', 'acqua_g': '69.6', 'energia_kcal_100g': 165, 'energia_kJ_100g': 692, 'proteine': 4.6, 'colesterolo_mg': 0.0, 'carboidrati_g': '13.6', 'amido_g': '11.1', 'zuccheri_solubili_g': 1.4, 'alcool_g': 0.0, 'fibra_g': '1.2'}
Errore: Nessun dato trovato per l'ingrediente 'Guanciale'
Dati grezzi per 'Pecorino romano': {'id_alimento': '166060', 'nome_alimento': 'Pecorino romano', 'categoria': 'Formaggi e latticini', 'parte_edibile': 1.0, 'porzione': '50 g', 'acqua_g': '31.9', 'energia_kcal_100g': 409, 'energia_kJ_100g': 1710, 'proteine': 26.0, 'colesterolo_mg': 90.0, 'carboidrati_g': '1.8', 'amido_g': 0, 'zuccheri_solubili_g': 1.8, 'alcool_g': 0.0, 'fibra_g': 0}

Nutrienti Totali per Spaghett

# CODICE PARZIALE - Estrazione dei valori nutrizionali da USDA a partire dal nome del cibo/ingrediente e creazione/update della tabella 'Ingredienti' del DB sqlite ricetta_prova.db

In [ ]:
import requests
import sqlite3
import os
import re
import logging
from typing import Optional, Dict, Tuple
from rapidfuzz import process, fuzz
import pandas as pd

# Configurazione del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class NutritionDataManager:
    def __init__(self, db_name: str, api_key: str):
        self.db_name = db_name
        self.api_key = api_key
        self.ensure_table_exists()
        self.cache_db = self.load_cache()

    def ensure_table_exists(self):
        """
        Assicura che la tabella 'ingredienti' esista nel database,
        creandola se necessario con le colonne di base.
        """
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        c.execute('''
            CREATE TABLE IF NOT EXISTS ingredienti (
                Nome_ingrediente TEXT PRIMARY KEY,
                Descrizione_USDA TEXT,
                USDA_match BOOLEAN,
                Similarity_score REAL,
                cibi_non_trovati BOOLEAN,
                Energy_KCAL REAL,
                Energy_KJ REAL
            )
        ''')
        conn.commit()
        conn.close()

    def load_cache(self) -> pd.DataFrame:
        """
        Carica i dati dalla tabella 'ingredienti' in un DataFrame pandas.
        """
        conn = sqlite3.connect(self.db_name)
        cache_db = pd.read_sql_query("SELECT * FROM ingredienti", conn)
        conn.close()
        return cache_db

    def get_nutritional_data(self, ingredient: str) -> Tuple[Optional[Dict], Optional[str], bool, float]:
        """
        Recupera i dati nutrizionali per un ingrediente, prima cercando nella cache locale,
        poi facendo una chiamata API a USDA se necessario.
        """
        cache_match = self.cache_db[self.cache_db['Nome_ingrediente'].str.lower() == ingredient.lower()]
        if not cache_match.empty:
            logging.info(f"Ingrediente '{ingredient}' trovato nella cache locale")
            return cache_match.iloc[0].to_dict(), None, cache_match.iloc[0]['USDA_match'], cache_match.iloc[0]['Similarity_score']

        search_url = f"https://api.nal.usda.gov/fdc/v1/foods/search?query={ingredient}&api_key={self.api_key}"
        try:
            response = requests.get(search_url)
            response.raise_for_status()
            data = response.json()
        except requests.RequestException as e:
            logging.error(f"Errore nella richiesta API per '{ingredient}': {str(e)}")
            return None, search_url, False, 0

        if 'foods' in data and data['foods']:
            exact_match = next((food for food in data['foods'] if food['description'].lower() == ingredient.lower()), None)
            if exact_match:
                processed_data = self.process_food_data(exact_match, ingredient)
                self.add_to_cache(processed_data, ingredient, True, 100)
                return processed_data, search_url, True, 100

            food_descriptions = [food['description'] for food in data['foods']]
            best_match = process.extractOne(ingredient, food_descriptions, scorer=fuzz.WRatio)
            if best_match and best_match[1] >= 80:
                fuzzy_match = next(food for food in data['foods'] if food['description'] == best_match[0])
                processed_data = self.process_food_data(fuzzy_match, ingredient)
                self.add_to_cache(processed_data, ingredient, True, best_match[1])
                return processed_data, search_url, True, best_match[1]

        logging.warning(f"Ingrediente '{ingredient}' non trovato nel DB USDA")
        not_found_data = {
            'Nome_ingrediente': ingredient,
            'Descrizione_USDA': None,
            'USDA_match': False,
            'Similarity_score': 0,
            'cibi_non_trovati': True
        }
        self.add_to_cache(not_found_data, ingredient, False, 0)
        return not_found_data, search_url, False, 0

    def process_food_data(self, food: Dict, original_name: str) -> Dict:
        """
        Elabora i dati grezzi dell'alimento restituiti dall'API USDA,
        strutturandoli nel formato richiesto per il database locale.
        """
        nutrients = food.get('foodNutrients', [])
        processed_data = {
            'Nome_ingrediente': original_name,
            'Descrizione_USDA': food['description'],
            'Energy_KCAL': None,
            'Energy_KJ': None
        }
        for nutrient in nutrients:
            nutrient_name = nutrient['nutrientName']
            nutrient_amount = nutrient['value']
            nutrient_unit = nutrient.get('unitName', '')

            if nutrient_name == 'Energy':
                if nutrient_unit == 'KCAL':
                    processed_data['Energy_KCAL'] = nutrient_amount
                elif nutrient_unit == 'kJ':
                    processed_data['Energy_KJ'] = nutrient_amount
            else:
                # Creiamo il nome della colonna includendo l'unità di misura
                nutrient_key = f"{re.sub(r'[^a-zA-Z0-9]+', '_', nutrient_name)}_{nutrient_unit}"
                # Salviamo solo il valore numerico
                processed_data[nutrient_key] = nutrient_amount


        return processed_data

    def add_to_cache(self, data: Dict, ingredient: str, usda_match: bool, similarity_score: float):
        """
        Aggiunge o aggiorna i dati nutrizionali di un ingrediente nel database locale,
        creando nuove colonne se necessario.
        """
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()

        existing_columns = [column[1] for column in c.execute("PRAGMA table_info(ingredienti)").fetchall()]
        for column in data.keys():
            if column not in existing_columns:
                c.execute(f"ALTER TABLE ingredienti ADD COLUMN {column} REAL")

        columns = ', '.join(data.keys())
        placeholders = ', '.join(['?' for _ in data])
        query = f"INSERT OR REPLACE INTO ingredienti ({columns}, USDA_match, Similarity_score, cibi_non_trovati) VALUES ({placeholders}, ?, ?, ?)"

        values = list(data.values()) + [usda_match, similarity_score, data.get('cibi_non_trovati', False)]
        c.execute(query, values)

        conn.commit()
        conn.close()

        self.cache_db = self.load_cache()

    def update_nutritional_values_in_db(self, food_name: str, nutritional_values: Dict):
        """
        Aggiorna i valori nutrizionali di un alimento nel database locale.
        """
        self.add_to_cache(nutritional_values, food_name, True, 100)

def main():
    """
    Funzione principale per testare la classe NutritionDataManager.
    """
    api_key = 'lRRUhBqakDfuwdjbdfEEck02bklmh6ml32q5ANxp'
    db_name = 'ricette_prova.db'
    nutrition_manager = NutritionDataManager(db_name, api_key)

    food_name = 'Eggs, Grade A, Large, egg yolk'
    nutritional_data, search_url, usda_match, similarity_score = nutrition_manager.get_nutritional_data(food_name)

    if nutritional_data:
        print(f"Dati nutrizionali per '{food_name}':")
        print(f"Descrizione USDA: {nutritional_data['Descrizione_USDA']}")
        print(f"Corrispondenza USDA: {'Sì' if usda_match else 'No'}")
        print(f"Punteggio di similarità: {similarity_score}")
        print(f"Energia (KCAL): {nutritional_data['Energy_KCAL']}")
        print(f"Energia (KJ): {nutritional_data['Energy_KJ']}")
        for key, value in nutritional_data.items():
            if key not in ['Nome_ingrediente', 'Descrizione_USDA', 'Energy_KCAL', 'Energy_KJ']:
                print(f"{key}: {value}")
    else:
        print(f"Impossibile trovare dati nutrizionali per '{food_name}'")

if __name__ == "__main__":
    main()

Dati nutrizionali per 'Eggs, Grade A, Large, egg yolk':
Descrizione USDA: Eggs, Grade A, Large, egg yolk
Corrispondenza USDA: Sì
Punteggio di similarità: 100
Energia (KCAL): 334
Energia (KJ): 1400.0
Lycopene_UG: 2.0
Selenium_Se_UG: 55.8
Total_lipid_fat__G: 28.8
Water_G: 52.1
Protein_G: 16.2
Ash_G: 1.89
Riboflavin_MG: 0.467
Carbohydrate_by_difference_G: 1.02
Nitrogen_G: 2.59
Tryptophan_G: 0.174
Tyrosine_G: 0.684
Valine_G: 0.846
Threonine_G: 0.789
Proline_G: 0.748
Serine_G: 1.29
Phenylalanine_G: 0.673
Methionine_G: 0.398
Leucine_G: 1.35
Lysine_G: 1.2
Histidine_G: 0.369
Isoleucine_G: 0.766
Glutamic_acid_G: 1.99
Glycine_G: 0.47
Aspartic_acid_G: 1.5
Arginine_G: 1.12
Alanine_G: 0.769
Hydroxyproline_G: 0.0
Cysteine_G: 0.425
cis_beta_Carotene_UG: 0.0
trans_beta_Carotene_UG: 0.0
cis_Lycopene_UG: 0.0
trans_Lycopene_UG: 2.0
Lutein_UG: 612
Zeaxanthin_UG: 546
cis_Lutein_Zeaxanthin_UG: 121
Vitamin_E_alpha_tocopherol__MG: 5.64
Tocopherol_beta_MG: 1.02
Tocopherol_gamma_MG: 0.22
Tocotrienol_alpha_MG: 0

# INIZIO PROGRAMMMA PER CREAZIONE DB
.
.
.
.
.
.
.
.

# 1. Creazione link ricette e creazione file recipe_links.txt

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

def get_recipe_links_from_page(url):
    """Estrae i link delle ricette da una singola pagina."""
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        titles = soup.find_all('h2', class_='gz-title')
        links = [title.find('a')['href'] for title in titles if title.find('a')]
        return links
    else:
        print(f"Errore nella richiesta: {response.status_code}")
        return []

def get_all_recipe_links(base_url, start_page, end_page):
    """Estrae i link delle ricette da tutte le pagine specificate."""
    all_links = []
    for page_number in range(start_page, end_page + 1):
        url = f"{base_url}/page{page_number}/colazione/"  # <------------------ Modifica 'Dolci' per altre categorie'''
        print(f"Estrazione dei link dalla pagina: {url}")
        links = get_recipe_links_from_page(url)
        if not links:
            break
        all_links.extend(links)
    return all_links

def save_links_to_file(links, filename):
    """Salva i link in un file, evitando duplicati."""
    existing_links = set()
    if os.path.exists(filename):
        with open(filename, 'r') as file:
            existing_links = set(file.read().splitlines())
    new_links = set(links) - existing_links
    with open(filename, 'a') as file:
        for link in new_links:
            file.write(link + '\n')
    return len(new_links)

def get_user_input():
    """Ottiene l'intervallo di pagine da scaricare dall'utente."""
    while True:
        try:
            start_page = int(input("Inserisci il numero della pagina di inizio: "))
            end_page = int(input("Inserisci il numero della pagina di fine: "))
            if start_page > 0 and end_page >= start_page:
                return start_page, end_page
            else:
                print("Per favore, inserisci un intervallo valido.")
        except ValueError:
            print("Per favore, inserisci un numero valido.")

# URL di base della categoria
#base_url = "https://www.giallozafferano.it/ricette-cat"  #<--------------  modifica se necessario
base_url = "https://www.giallozafferano.it/ricerca-ricette" #<--------------serve solo per le colazioni!!! modifica se necessario

# Ottieni l'intervallo di pagine dall'utente
start_page, end_page = get_user_input()

# Estrai i link delle ricette dalle pagine specificate
recipe_links = get_all_recipe_links(base_url, start_page, end_page)

# Nome del file per salvare i link
filename = 'recipe_links.txt'

# Salva i link nel file, evitando duplicati, e ottieni il numero di nuovi link
new_links_count = save_links_to_file(recipe_links, filename)

# Stampa un messaggio con il numero di nuovi link salvati
print(f"{new_links_count} nuovi link salvati in '{filename}'")


Inserisci il numero della pagina di inizio: 1
Inserisci il numero della pagina di fine: 1
Estrazione dei link dalla pagina: https://www.giallozafferano.it/ricerca-ricette/page1/colazione/
15 nuovi link salvati in 'recipe_links.txt'


# 2.  Estrazione dei dati di TUTTE le ricette contenute nel file recipe_links.txt
*   salvataggio su file excel e sQlite
*   Le foto delle ricette vengono salvate nella cartella  IMAGES con lo stesso nome della ricette
* il link della foto è salvato nel DB


In [ ]:
!pip install requests beautifulsoup4 pandas openpyxl

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import sqlite3

# Definisci le colonne che saranno usate sia per Excel che per SQLite
COLUMNS = [
    'URL', 'Titolo', 'Presentazione', 'Calorie_per_porzione', 'Difficolta',
    'Dosi_per', 'Preparazione', 'Cottura', 'Costo', 'Ingredienti', 'Istruzioni',
    'Immagine_URL', 'Ingredients', 'Calorie_calcolate', 'Tipologia_dieta', 'Tipologia_piatti'
]

import requests
from bs4 import BeautifulSoup
import re

def scrape_recipe(url, tipologia_dieta, tipologia_piatti):
    """Estrae le informazioni di una ricetta da un URL dato."""
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    title = soup.find('h1', class_='gz-title-recipe')
    title = title.text.strip() if title else 'Non specificato'

    presentation_div = soup.find('div', class_='gz-content-recipe')
    presentation = presentation_div.find('p').text.strip() if presentation_div and presentation_div.find('p') else 'Non specificato'

    calories_div = soup.find('div', class_='gz-text-calories-total')
    calories = calories_div.find('span').text.strip() if calories_div else '0'
    calories = int(re.search(r'\d+', calories).group()) if re.search(r'\d+', calories) else 0

    difficulty = servings = preparation_time = cooking_time = cost = 'Non specificato'

    for span in soup.find_all('span', class_='gz-name-featured-data'):
        if 'Difficoltà' in span.text:
            difficulty = span.find('strong').text.strip()
        elif 'Dosi per' in span.text:
            servings = span.find('strong').text.strip()
            servings = int(re.search(r'\d+', servings).group()) if re.search(r'\d+', servings) else 0
        elif 'Preparazione' in span.text:
            preparation_time = span.find('strong').text.strip()
        elif 'Cottura' in span.text:
            cooking_time = span.find('strong').text.strip()
        elif 'Costo' in span.text:
            cost = span.find('strong').text.strip()

    ingredients = []
    for section in soup.find_all('dl', class_='gz-list-ingredients'):
        section_title = section.find('dt', class_='gz-title-ingredients')
        if section_title:
            ingredients.append(f"{section_title.text.strip()}:")
        for ingredient in section.find_all('dd', class_='gz-ingredient'):
            ingredient_text = ' '.join(ingredient.stripped_strings)
            ingredient_text = re.sub(r'\s+', ' ', ingredient_text).strip()
            ingredients.append(ingredient_text)

    preparation = []
    for step in soup.find_all('div', class_='gz-content-recipe-step'):
        p_tag = step.find('p')
        if p_tag:
            preparation.append(p_tag.text.strip())

    image_url = extract_image_url(soup)

    return {
        'URL': url,
        'Titolo': title,
        'Presentazione': presentation,
        'Calorie_per_porzione': calories,
        'Difficolta': difficulty,
        'Dosi_per': servings,
        'Preparazione': preparation_time,
        'Cottura': cooking_time,
        'Costo': cost,
        'Ingredienti': '\n'.join(ingredients),
        'Istruzioni': '\n'.join(preparation),
        'Immagine_URL': image_url,
        'Ingredients': '',  # Colonna vuota per compatibilità
        'Calorie_calcolate': 0,  # Valore di default
        'Tipologia_dieta': tipologia_dieta,
        'Tipologia_piatti': tipologia_piatti
    }

def extract_image_url(soup):
    """Estrae l'URL dell'immagine dalla pagina."""
    picture_tag = soup.find('picture')
    if picture_tag:
        img_tag = picture_tag.find('img')
        if img_tag and 'src' in img_tag.attrs:
            return img_tag['src']

    image_div = soup.find('div', class_='gz-featured-image-video')
    if image_div:
        picture_tag = image_div.find('picture')
        if picture_tag:
            img_tag = picture_tag.find('img')
            if img_tag and 'src' in img_tag.attrs:
                return img_tag['src']

    return None

def save_to_excel(recipe_data, filename='ricette.xlsx'):
    """Salva o aggiorna i dati della ricetta in un file Excel."""
    df = pd.DataFrame([recipe_data], columns=COLUMNS)

    if os.path.exists(filename):
        existing_df = pd.read_excel(filename)
        if recipe_data['URL'] not in existing_df['URL'].values:
            updated_df = pd.concat([existing_df, df], ignore_index=True)
            updated_df.to_excel(filename, index=False, columns=COLUMNS)
            print(f"Nuova ricetta aggiunta al file esistente: {filename}")
        else:
            print(f"La ricetta con URL {recipe_data['URL']} esiste già nel file. Nessuna modifica apportata.")
    else:
        df.to_excel(filename, index=False, columns=COLUMNS)
        print(f"Nuovo file creato: {filename}")

def create_database():
    """Crea il database SQLite se non esiste."""
    conn = sqlite3.connect('ricette.db')
    c = conn.cursor() # Creiamo un cursore per eseguire le query SQL
    c.execute('''CREATE TABLE IF NOT EXISTS ricette
                 (URL TEXT PRIMARY KEY,
                  Titolo TEXT,
                  Presentazione TEXT,
                  Calorie_per_porzione INTEGER,
                  Difficolta TEXT,
                  Dosi_per INTEGER,
                  Preparazione TEXT,
                  Cottura TEXT,
                  Costo TEXT,
                  Ingredienti TEXT,
                  Ingredients TEXT,
                  Istruzioni TEXT,
                  Immagine_URL TEXT,
                  Calorie_calcolate INTEGER,
                  Tipologia_dieta TEXT,
                  Tipologia_piatti TEXT)
              ''')
    conn.commit()
    conn.close()

def save_to_database(recipe_data):
    """Salva o aggiorna i dati della ricetta nel database SQLite."""
    conn = sqlite3.connect('ricette.db')
    c = conn.cursor()  # Creiamo un cursore per eseguire le query SQL

    # Modifica della query per includere tutti i campi esplicitamente
    c.execute('''INSERT OR REPLACE INTO ricette
                 (URL, Titolo, Presentazione, Calorie_per_porzione, Difficolta, Dosi_per,
                  Preparazione, Cottura, Costo, Ingredienti, Ingredients, Istruzioni, Immagine_URL, Calorie_calcolate,
                  Tipologia_dieta, Tipologia_piatti)
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
              (recipe_data['URL'], recipe_data['Titolo'], recipe_data['Presentazione'],
               recipe_data['Calorie_per_porzione'], recipe_data['Difficolta'], recipe_data['Dosi_per'],
               recipe_data['Preparazione'], recipe_data['Cottura'], recipe_data['Costo'],
               recipe_data['Ingredienti'], recipe_data.get('Ingredients', ''), recipe_data['Istruzioni'],
               recipe_data['Immagine_URL'], recipe_data.get('Calorie_calcolate', 0),
               recipe_data['Tipologia_dieta'], recipe_data['Tipologia_piatti']))

    conn.commit()
    conn.close()
    print(f"Ricetta salvata/aggiornata nel database: {recipe_data['Titolo']}")

def read_links_from_file(filename):
    """Legge i link delle ricette da un file di testo."""
    with open(filename, 'r') as file:
        links = [line.strip() for line in file if line.strip()]
    return links

def download_image(image_url, recipe_title):
    """Scarica l'immagine della ricetta."""
    if not image_url:
        print(f"Nessun URL dell'immagine fornito per la ricetta: {recipe_title}")
        return None

    try:
        response = requests.get(image_url, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Errore nel download dell'immagine per {recipe_title}: {str(e)}")
        return None

    if response.status_code == 200:
        if not os.path.exists('images'):
            os.makedirs('images')

        filename = re.sub(r'[^\w\-_\. ]', '_', recipe_title)
        filename = re.sub(r'\s+', '_', filename).strip().lower()
        filename = f"images/{filename}.jpg"

        try:
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Immagine salvata con successo: {filename}")
            return filename
        except IOError as e:
            print(f"Errore nel salvare l'immagine per {recipe_title}: {str(e)}")
            return None
    else:
        print(f"Impossibile scaricare l'immagine per {recipe_title}. Codice di stato: {response.status_code}")
        return None

def recipe_exists_in_db(url):
    """Verifica se una ricetta esiste già nel database."""
    conn = sqlite3.connect('ricette.db')
    c = conn.cursor()
    c.execute("SELECT 1 FROM ricette WHERE URL = ?", (url,))
    exists = c.fetchone() is not None
    conn.close()
    return exists

def main():
    create_database()
    input_file = 'recipe_links.txt'
    links = read_links_from_file(input_file)
    tipologia_dieta = input("Inserisci la tipologia di dieta per le ricette(solo lettere minuscole): ")
    tipologia_piatti = input("Inserisci la tipologia di piatti per le ricette:(colazione/pranzo/cena) ")

    for url in links:
        if not recipe_exists_in_db(url):
            recipe_data = scrape_recipe(url, tipologia_dieta, tipologia_piatti)
            save_to_database(recipe_data)
            save_to_excel(recipe_data)
            download_image(recipe_data['Immagine_URL'], recipe_data['Titolo'])
        else:
            print(f"La ricetta con URL {url} esiste già nel database.")

if __name__ == "__main__":
    main()


Ricetta salvata/aggiornata nel database: Torta di carote
Nuovo file creato: ricette.xlsx
Immagine salvata con successo: images/torta_di_carote.jpg
Ricetta salvata/aggiornata nel database: Torta al cioccolato
Nuova ricetta aggiunta al file esistente: ricette.xlsx
Immagine salvata con successo: images/torta_al_cioccolato.jpg
Ricetta salvata/aggiornata nel database: Muffin con gocce di cioccolato
Nuova ricetta aggiunta al file esistente: ricette.xlsx
Immagine salvata con successo: images/muffin_con_gocce_di_cioccolato.jpg
Ricetta salvata/aggiornata nel database: Pasta per il pane
Nuova ricetta aggiunta al file esistente: ricette.xlsx
Immagine salvata con successo: images/pasta_per_il_pane.jpg
Ricetta salvata/aggiornata nel database: Torta tenerina
Nuova ricetta aggiunta al file esistente: ricette.xlsx
Immagine salvata con successo: images/torta_tenerina.jpg
Ricetta salvata/aggiornata nel database: Torta 7 vasetti
Nuova ricetta aggiunta al file esistente: ricette.xlsx
Immagine salvata con 

In [ ]:
import pandas as pd

# Leggi il file Excel
df = pd.read_excel('ricette.xlsx')

# Mostra le prime 10 righe
print(df.head(20))

                                                 URL  \
0  https://ricette.giallozafferano.it/Torta-tener...   
1   https://ricette.giallozafferano.it/Tiramisu.html   
2  https://ricette.giallozafferano.it/Crepes-dolc...   
3  https://ricette.giallozafferano.it/Spaghetti-a...   
4  https://ricette.giallozafferano.it/Pancakes-al...   

                                Titolo  \
0                       Torta tenerina   
1                             Tiramisù   
2  Crepe dolci e salate (ricetta base)   
3             Spaghetti alla Carbonara   
4        Pancake allo sciroppo d'acero   

                                       Presentazione  Calorie_per_porzione  \
0  La torta tenerina è un dolce tipico della citt...                   395   
1  Il tiramisù è uno dei capisaldi della cucina i...                   670   
2  Che si tratti di un’amorevole mamma che prepar...                   269   
3  Dove sono nati gli spaghetti alla carbonara? I...                   680   
4  Non c’è colazione

# 3. Traduzione automatica colonna Ingredienti e creazione colonna Ingredients

In [ ]:
!pip install pandas openpyxl deep-translator
!pip install googletrans

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 760.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 980.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15718 sha256=e41fa6cfb2479f5f136e7a8db06dbae83d5b9fdeebff7322eac505b0d22a57e1
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uni

In [14]:
import pandas as pd
from deep_translator import GoogleTranslator
import sqlite3
import time
from functools import partial
from concurrent.futures import ThreadPoolExecutor, as_completed

# Carica il file Excel
df = pd.read_excel('ricette.xlsx')

# Inizializza il traduttore
translator = GoogleTranslator(source='it', target='en')

# Funzione per tradurre il testo con gestione degli errori e ritardi
def translate_text(text, max_retries=3, initial_delay=1):
    for attempt in range(max_retries):
        try:
            return translator.translate(text)
        except Exception as e:
            print(f"Errore durante la traduzione: {e}. Tentativo {attempt + 1} di {max_retries}")
            if attempt < max_retries - 1:
                time.sleep(initial_delay * (1 ** attempt))  # Backoff esponenziale
    return text  # Ritorna il testo originale se tutti i tentativi falliscono

# Funzione per tradurre un batch di testi
def translate_batch(texts):
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(translate_text, text) for text in texts]
        return [future.result() for future in as_completed(futures)]

# Identifica le righe da tradurre
mask = df['Ingredients'].isna() | (df['Ingredients'] == '')
texts_to_translate = df.loc[mask, 'Ingredienti'].tolist()

# Traduci in batch
translated_texts = translate_batch(texts_to_translate)

# Aggiorna il DataFrame
df.loc[mask, 'Ingredients'] = translated_texts

# Salva il DataFrame aggiornato nel file Excel esistente
df.to_excel('ricette.xlsx', index=False)

# Connessione al database SQLite e aggiornamento
with sqlite3.connect('ricette.db') as conn:
    c = conn.cursor()

    # Prepara la query una volta
    update_query = "UPDATE ricette SET Ingredients = ? WHERE URL = ?"

    # Esegui l'aggiornamento in blocco
    c.executemany(update_query, df[['Ingredients', 'URL']].values.tolist())

    # Le modifiche vengono salvate automaticamente quando il contesto viene chiuso

print("Traduzione completata. Il file Excel e il database sono stati aggiornati.")

Traduzione completata. Il file Excel e il database sono stati aggiornati.


# 4. Normalizzazione dei dati di Ingredienti ed Ingredients e creazione di Ingredienti_JSON + Ingredients_JSON. Con questo Update sarà facile gestire i dati relativi agli ingredienti.

In [ ]:
import sqlite3
import json
import re

def parse_ingredients_italian(text):
    if not text or text.strip() == '':
        return {}

    # Rimuoviamo le sezioni iniziali come "Ingredienti per..."
    ingredients_text = re.sub(r'^.*?(?:Ingredienti|Per).*?:', '', text, flags=re.DOTALL)

    # Dividiamo il testo in sezioni
    sections = re.split(r'\n(?=(?:per\s+\w+:|Per\s+\w+:|Per\s+(?:la|il|le|i|lo)\s+\w+:))', ingredients_text)

    ingredients = {}
    current_section = 'main'

    for section in sections:
        section = section.strip()
        if section.lower().startswith(('per ', 'per la ', 'per il ', 'per le ', 'per i ', 'per lo ')):
            current_section = section.split(':')[0].strip().lower()
            section = re.sub(r'^per.*?:', '', section, flags=re.IGNORECASE)

        lines = section.strip().split('\n')
        for line in lines:
            if line.strip():
                ingredient, quantity, unit = parse_ingredient_line_italian(line)
                if ingredient:
                    if current_section not in ingredients:
                        ingredients[current_section] = {}
                    ingredients[current_section][ingredient] = {'quantity': quantity, 'unit': unit}

    return ingredients

def parse_ingredient_line_italian(line):
    line = line.strip()

    # Gestione di q.b.
    if 'q.b.' in line.lower():
        ingredient = re.sub(r'\s*q\.b\.\s*$', '', line, flags=re.IGNORECASE).strip()
        return ingredient.lower(), 'q.b.', ''

    # Rimuoviamo le parentesi e il loro contenuto, ma conserviamo l'informazione
    parentheses_content = re.findall(r'\(([^)]*)\)', line)
    line = re.sub(r'\([^)]*\)', '', line).strip()

    # Regex aggiornata per catturare vari formati
    match = re.match(r'^(.*?)\s*((?:\d+(?:[.,]\d+)?)?(?:\s*[a-zA-Z.]+)?)\s*$', line)

    if match:
        ingredient, quantity_unit = match.groups()
        ingredient = ingredient.strip().lower()

        # Gestione delle uova e tuorli
        if 'uova' in ingredient or 'uovo' in ingredient or 'tuorli' in ingredient or 'tuorlo' in ingredient:
            egg_match = re.search(r'(\d+)', quantity_unit)
            if egg_match:
                return ingredient, int(egg_match.group(1)), 'pezzi'
            else:
                # Se non c'è un numero esplicito, usiamo la quantità in grammi
                quantity_match = re.match(r'([\d.,]+)\s*(.*)$', quantity_unit)
                if quantity_match:
                    quantity, unit = quantity_match.groups()
                    return ingredient, float(quantity.replace(',', '.')), unit.strip() or 'g'

        # Gestione di ingredienti come "Scorza di limone 1"
        if quantity_unit.strip() == '1':
            return ingredient, 1, 'pezzo'

        # Separazione di quantità e unità
        quantity_match = re.match(r'([\d.,]+)\s*(.*)$', quantity_unit)
        if quantity_match:
            quantity, unit = quantity_match.groups()
            try:
                quantity = float(quantity.replace(',', '.'))
            except ValueError:
                quantity = quantity.strip()
            unit = unit.strip() or 'g'  # Se l'unità è vuota, assumiamo grammi
        else:
            quantity, unit = quantity_unit, ''

        # Aggiungiamo l'informazione tra parentesi all'ingrediente
        if parentheses_content:
            ingredient += f" ({', '.join(parentheses_content)})"

        return ingredient, quantity, unit

    # Se non c'è una quantità chiara, restituiamo l'ingrediente così com'è
    return line.strip().lower(), None, ''

def parse_ingredients_english(text):
    if not text or text.strip() == '':
        return {}

    # Split the text into sections
    sections = re.split(r'\n(?=(?:Ingredients:|To cook:|to garnish:|For the .*?:))', text, flags=re.IGNORECASE)

    ingredients = {}
    current_section = 'main'

    for section in sections:
        section = section.strip()
        if section.lower().startswith('ingredients:'):
            current_section = 'main'
            section = re.sub(r'^ingredients:', '', section, flags=re.IGNORECASE)
        elif section.lower().startswith('to cook:'):
            current_section = 'to_cook'
            section = re.sub(r'^to cook:', '', section, flags=re.IGNORECASE)
        elif section.lower().startswith('to garnish:'):
            current_section = 'to_garnish'
            section = re.sub(r'^to garnish:', '', section, flags=re.IGNORECASE)
        elif section.lower().startswith('for the'):
            current_section = section.split(':')[0].strip().lower()
            section = re.sub(r'^for the.*?:', '', section, flags=re.IGNORECASE)

        lines = section.strip().split('\n')
        for line in lines:
            if line.strip():
                ingredient, quantity, unit = parse_ingredient_line_english(line)
                if ingredient:
                    if current_section not in ingredients:
                        ingredients[current_section] = {}
                    ingredients[current_section][ingredient] = {'quantity': quantity, 'unit': unit}

    return ingredients

def parse_ingredient_line_english(line):
    line = line.strip()

    # Handle "to taste" cases
    if line.lower().endswith('to taste'):
        ingredient = re.sub(r'\s+to taste$', '', line, flags=re.IGNORECASE).strip()
        return ingredient.lower(), 'to taste', ''

    # Remove parentheses and their content
    line = re.sub(r'\([^)]*\)', '', line).strip()

    # Match patterns like "320 g Greek yogurt", "Greek yogurt 320 g", "1 tbsp Extra virgin olive oil"
    match = re.match(r'^([\d./]+)\s*([a-zA-Z]*)\s+(.+)$|^(.+?)\s+([\d./]+)\s*([a-zA-Z]*)$|^([\d./]+)\s+([a-zA-Z]+)\s+(.+)$', line)

    if match:
        if match.group(1):  # First pattern matched
            quantity, unit, ingredient = match.group(1, 2, 3)
        elif match.group(4):  # Second pattern matched
            ingredient, quantity, unit = match.group(4, 5, 6)
        else:  # Third pattern matched
            quantity, unit, ingredient = match.group(7, 8, 9)

        ingredient = ingredient.strip().lower()

        # Handle special cases
        if 'egg' in ingredient:
            return ingredient, int(quantity) if quantity.isdigit() else quantity, 'pieces'

        try:
            quantity = float(quantity.replace(',', '.'))
        except ValueError:
            # If conversion fails, keep it as a string
            pass

        return ingredient, quantity, unit.strip() if unit else ''

    # Handle cases like "1 lemon" or "Salt 1 pinch"
    match = re.match(r'^(\d+)\s+(.+)$|^(.+?)\s+(\d+)\s+(.+)$', line)
    if match:
        if match.group(1):  # First pattern matched
            quantity, ingredient = match.group(1, 2)
            unit = ''
        else:  # Second pattern matched
            ingredient, quantity, unit = match.group(3, 4, 5)

        return ingredient.strip().lower(), int(quantity), unit.strip()

    # If no quantity is found, return the ingredient as is
    return line.strip().lower(), None, ''

# Test the function with multiple inputs
test_inputs = [
    '''Small tortillas 4
    Eggs 4
    Precooked black beans 200 g
    Cherry tomatoes (red, green, yellow) 200 g
    Red onions 100 g
    Cheddar 80g
    Feta 20 g
    Long fresh chili pepper 1
    Extra virgin olive oil 1 tbsp
    For the guacamole:
    Avocado 2
    Lime (juice and zest) 1
    Extra virgin olive oil 1 tbsp
    Fine salt 1 teaspoon
    Black pepper 1 tsp''',

    '''Oat flakes 160 g
    Whole milk 200 g
    Water 160 g
    Wildflower honey 1 tbsp
    Salt to taste
    to garnish:
    Natural plain yogurt 2 tablespoons
    Strawberries to taste
    Dark chocolate flakes to taste
    Maple syrup to taste'''
]

for i, test_input in enumerate(test_inputs, 1):
    print(f"\nTest Input {i}:")
    result = parse_ingredients_english(test_input)
    print(json.dumps(result, indent=2))

def ensure_column_exists(cursor, table_name, column_name):
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = [column[1] for column in cursor.fetchall()]
    if column_name not in columns:
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} TEXT")
        print(f"Colonna '{column_name}' creata nella tabella '{table_name}'.")

def update_database():
    conn = sqlite3.connect('ricette.db')  # Connettiamo al database
    cursor = conn.cursor()  # Creiamo un cursore per eseguire le query SQL

    ensure_column_exists(cursor, 'ricette', 'Ingredients_JSON')
    ensure_column_exists(cursor, 'ricette', 'Ingredienti_JSON')

    cursor.execute("""
        SELECT URL, Ingredients, Ingredienti, Ingredients_JSON, Ingredienti_JSON
        FROM ricette
        WHERE (Ingredients_JSON IS NULL) OR (Ingredienti_JSON IS NULL)
    """)
    rows = cursor.fetchall()

    processed_rows = 0
    updated_rows = 0

    for url, ingredients_en, ingredients_it, ingredients_json_en, ingredients_json_it in rows:
        processed_rows += 1
        row_updated = False

        if ingredients_en and ingredients_json_en is None:
            structured_ingredients_en = parse_ingredients_english(ingredients_en)
            if structured_ingredients_en:
                structured_ingredients_en_json = json.dumps(structured_ingredients_en)
                cursor.execute("UPDATE ricette SET Ingredients_JSON = ? WHERE URL = ?",
                              (structured_ingredients_en_json, url))
                row_updated = True

        if ingredients_it and ingredients_json_it is None:
            structured_ingredients_it = parse_ingredients_italian(ingredients_it)
            if structured_ingredients_it:
                structured_ingredients_it_json = json.dumps(structured_ingredients_it)
                cursor.execute("UPDATE ricette SET Ingredienti_JSON = ? WHERE URL = ?",
                              (structured_ingredients_it_json, url))
                row_updated = True

        if row_updated:
            updated_rows += 1

    conn.commit()  # Salviamo le modifiche nel database
    conn.close()  # Chiudiamo la connessione al database

    return processed_rows, updated_rows

# Eseguiamo l'aggiornamento del database
processed, updated = update_database()
#print(f"Aggiornamento del database completato.")
#print(f"Righe processate: {processed}")
#print(f"Righe effettivamente aggiornate: {updated}")


Test Input 1:
{
  "main": {
    "small tortillas": {
      "quantity": 4.0,
      "unit": ""
    },
    "eggs": {
      "quantity": 4,
      "unit": "pieces"
    },
    "precooked black beans": {
      "quantity": 200.0,
      "unit": "g"
    },
    "cherry tomatoes": {
      "quantity": 200.0,
      "unit": "g"
    },
    "red onions": {
      "quantity": 100.0,
      "unit": "g"
    },
    "cheddar": {
      "quantity": 80.0,
      "unit": "g"
    },
    "feta": {
      "quantity": 20.0,
      "unit": "g"
    },
    "long fresh chili pepper": {
      "quantity": 1.0,
      "unit": ""
    },
    "extra virgin olive oil": {
      "quantity": 1.0,
      "unit": "tbsp"
    },
    "for the guacamole:": {
      "quantity": null,
      "unit": ""
    },
    "avocado": {
      "quantity": 2.0,
      "unit": ""
    },
    "lime": {
      "quantity": 1.0,
      "unit": ""
    },
    "fine salt": {
      "quantity": 1.0,
      "unit": "teaspoon"
    },
    "black pepper": {
      "quantity": 1

# 4.0 JSON nuovo

In [15]:
import sqlite3
import json
import re

import re

def parse_ingredients_italian(text):
    if not text or text.strip() == '':
        return {}

    ingredients_text = re.sub(r'^.*?(?:Ingredienti|Per).*?:', '', text, flags=re.DOTALL)
    lines = ingredients_text.split('\n')

    ingredients = {}
    for line in lines:
        if line.strip():
            ingredient, quantity, unit = parse_ingredient_line_italian(line)
            if ingredient and quantity is not None:
                ingredients[ingredient] = quantity

    return ingredients

def parse_ingredient_line_italian(line):
    line = line.strip().lower()

    # Rimuovi frasi non necessarie
    line = re.sub(r'(per una teglia di circa|per uno stampo del diametro di|bicarbonato|per dolci|sale fino|freddo|dal frigo|a temperatura ambiente|in polvere per dolci|per circa)\s*', '', line)

    if 'q.b.' in line:
        ingredient = re.sub(r'\s*q\.b\.\s*$', '', line, flags=re.IGNORECASE).strip()
        return ingredient, 'q.b.', ''

    match = re.match(r'^([\d.,]+)\s*([a-z]*)\s*(.+)$|^(.+?)\s+([\d.,]+)\s*([a-z]*)$', line)

    if match:
        if match.group(1):
            quantity, unit, ingredient = match.group(1, 2, 3)
        else:
            ingredient, quantity, unit = match.group(4, 5, 6)

        ingredient = ingredient.strip()

        if 'uova' in ingredient or 'uovo' in ingredient:
            return ingredient, int(float(quantity.replace(',', '.'))) * 60, 'g'  # Assumendo 60g per uovo

        try:
            quantity = float(quantity.replace(',', '.'))
        except ValueError:
            return None, None, None

        if not unit:
            unit = 'g'

        return ingredient, quantity, unit

    return None, None, None

def parse_ingredients_english(text):
    if not text or text.strip() == '':
        return {}

    ingredients_text = re.sub(r'^.*?(?:Ingredients).*?:', '', text, flags=re.DOTALL)
    lines = ingredients_text.split('\n')

    ingredients = {}
    for line in lines:
        if line.strip():
            ingredient, quantity, unit = parse_ingredient_line_english(line)
            if ingredient and quantity is not None:
                ingredients[ingredient] = quantity

    return ingredients

def parse_ingredient_line_english(line):
    line = line.strip().lower()

    # Rimuovi frasi non necessarie
    line = re.sub(r'(for a loaf of approximately|for a mold with a diameter of|bicarbonate|for cake|salt up to|cold|from the fridge|at room temperature|powder for cakes|for about)\s*', '', line)

    if 'to taste' in line:
        ingredient = re.sub(r'\s*to taste\s*$', '', line, flags=re.IGNORECASE).strip()
        return ingredient, 'to taste'

    # Gestione speciale per le uova
    egg_match = re.match(r'(eggs?)\s*\((?:about\s*)?(\d+)\s*(small|medium|large)?\)', line)
    if egg_match:
        _, quantity, size = egg_match.groups()
        quantity = int(quantity)
        if size:
            if size == 'small':
                return 'Eggs, Grade A, Large, egg whole', quantity * 40
            elif size == 'medium':
                return 'Eggs, Grade A, Large, egg whole', quantity * 50
            elif size == 'large':
                return 'Eggs, Grade A, Large, egg whole', quantity * 60
        return 'Eggs, Grade A, Large, egg whole', quantity * 50  # Default to medium if size not specified

    # Gestione generale per gli altri ingredienti
    match = re.match(r'^([\d./]+)\s*([a-z]*)\s*(.+)$|^(.+?)\s+([\d./]+)\s*([a-z]*)$', line)

    if match:
        if match.group(1):
            quantity, _, ingredient = match.group(1, 2, 3)
        else:
            ingredient, quantity, _ = match.group(4, 5, 6)

        ingredient = ingredient.strip()

        try:
            if '/' in quantity:
                num, denom = quantity.split('/')
                quantity = float(num) / float(denom)
            else:
                quantity = float(quantity)
        except ValueError:
            return None, None

        return ingredient, quantity

    # Gestione per 'eggs' senza specifiche
    if line == 'eggs':
        return 'Eggs, Grade A, Large, egg whole', 50  # Assumendo un uovo medio

    return None, None

def parse_ingredients(text, language='en'):
    if language.lower() == 'it':
        return parse_ingredients_italian(text)
    else:
        return parse_ingredients_english(text)

def ensure_column_exists(cursor, table_name, column_name):
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = [column[1] for column in cursor.fetchall()]
    if column_name not in columns:
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} TEXT")
        print(f"Colonna '{column_name}' creata nella tabella '{table_name}'.")

def update_database():
    conn = sqlite3.connect('ricette.db')
    cursor = conn.cursor()

    ensure_column_exists(cursor, 'ricette', 'Ingredients_JSON')
    ensure_column_exists(cursor, 'ricette', 'Ingredienti_JSON')

    cursor.execute("""
        SELECT URL, Ingredients, Ingredienti, Ingredients_JSON, Ingredienti_JSON
        FROM ricette
        WHERE (Ingredients_JSON IS NULL) OR (Ingredienti_JSON IS NULL)
    """)
    rows = cursor.fetchall()

    processed_rows = 0
    updated_rows = 0

    for url, ingredients_en, ingredients_it, ingredients_json_en, ingredients_json_it in rows:
        processed_rows += 1
        row_updated = False

        if ingredients_en and ingredients_json_en is None:
            structured_ingredients_en = parse_ingredients_english(ingredients_en)
            if structured_ingredients_en:
                structured_ingredients_en_json = json.dumps(structured_ingredients_en)
                cursor.execute("UPDATE ricette SET Ingredients_JSON = ? WHERE URL = ?",
                              (structured_ingredients_en_json, url))
                row_updated = True

        if ingredients_it and ingredients_json_it is None:
            structured_ingredients_it = parse_ingredients_italian(ingredients_it)
            if structured_ingredients_it:
                structured_ingredients_it_json = json.dumps(structured_ingredients_it)
                cursor.execute("UPDATE ricette SET Ingredienti_JSON = ? WHERE URL = ?",
                              (structured_ingredients_it_json, url))
                row_updated = True

        if row_updated:
            updated_rows += 1

    conn.commit()
    conn.close()

    return processed_rows, updated_rows

# Eseguiamo l'aggiornamento del database
processed, updated = update_database()
print(f"Aggiornamento del database completato.")
print(f"Righe processate: {processed}")
print(f"Righe effettivamente aggiornate: {updated}")

Colonna 'Ingredients_JSON' creata nella tabella 'ricette'.
Colonna 'Ingredienti_JSON' creata nella tabella 'ricette'.
Aggiornamento del database completato.
Righe processate: 15
Righe effettivamente aggiornate: 15


In [16]:
'''somma dei tempi di preparazione e cottura'''
import sqlite3
import re

def convert_time_to_minutes(time_str):
    if not time_str or time_str.lower() == 'n.d.':
        return 0
    total_minutes = 0
    parts = re.findall(r'(\d+)\s*([hm])', time_str.lower())
    for value, unit in parts:
        if unit == 'h':
            total_minutes += int(value) * 60
        elif unit == 'm':
            total_minutes += int(value)
    return total_minutes

def update_database_with_total_time():
    conn = sqlite3.connect('ricette.db')
    cursor = conn.cursor()

    # Verifica se la colonna esiste già
    cursor.execute("PRAGMA table_info(ricette)")
    columns = [column[1] for column in cursor.fetchall()]
    if 'Tempo_preparazione_totale' not in columns:
        cursor.execute("ALTER TABLE ricette ADD COLUMN Tempo_preparazione_totale INTEGER")

    # Aggiorna la tabella con il tempo totale
    cursor.execute("SELECT rowid, Preparazione, Cottura FROM ricette")
    for row in cursor.fetchall():
        rowid, prep_time, cook_time = row
        total_time = convert_time_to_minutes(prep_time) + convert_time_to_minutes(cook_time)
        cursor.execute("UPDATE ricette SET Tempo_preparazione_totale = ? WHERE rowid = ?", (total_time, rowid))

    conn.commit()
    print("Database aggiornato con successo. Colonna 'Tempo_preparazione_totale' aggiunta e popolata.")

    # Verifica alcuni risultati
    cursor.execute("SELECT Titolo, Preparazione, Cottura, Tempo_preparazione_totale FROM ricette LIMIT 5")
    print("\nEsempio di alcuni risultati:")
    for row in cursor.fetchall():
        print(row)

    conn.close()

# Esegui la funzione per aggiornare il database
update_database_with_total_time()

Database aggiornato con successo. Colonna 'Tempo_preparazione_totale' aggiunta e popolata.

Esempio di alcuni risultati:
('Torta di carote', '30 min', '50 min', 80)
('Torta al cioccolato', '15 min', '45 min', 60)
('Muffin con gocce di cioccolato', '15 min', '20 min', 35)
('Pasta per il pane', '20 min', '55 min', 75)
('Torta tenerina', '20 min', '25 min', 45)


# 4.1 OTTIMIZZAZIONE colonna 'Instruzioni' con eliminazione numeri superflui

In [17]:
import sqlite3
import re

def pulisci_spazi(testo):
    # Rimuove gli spazi prima della punteggiatura
    testo = re.sub(r'\s+([.,!?:;])', r'\1', testo)
    # Rimuove gli spazi doppi
    testo = re.sub(r'\s+', ' ', testo)
    return testo.strip()

def ottimizza_istruzioni(testo):
    # Pattern aggiornato per gestire i casi forniti
    pattern = r'\b(\d+(?:-\d+)?(?:/\d+)?)(?!\s*(?:cm|mm|m|kg|g|ml|l|minuti|minuto|ore|ora|°|x|\d+(?:,\d+)?cm))\b'

    def replace_func(match):
        # Se il numero fa parte di una frazione o di un intervallo, lo manteniamo
        if '/' in match.group(1) or '-' in match.group(1):
            return match.group(1)
        # Altrimenti, lo rimuoviamo
        return ''

    testo_ottimizzato = re.sub(pattern, replace_func, testo)
    # Applica la pulizia degli spazi
    testo_ottimizzato = pulisci_spazi(testo_ottimizzato)
    return testo_ottimizzato

def main():
    # Connessione al database
    conn = sqlite3.connect('ricette.db')
    cursor = conn.cursor()

    # Recupera tutte le righe dalla tabella Ricette
    cursor.execute("SELECT URL, Istruzioni FROM Ricette")
    righe = cursor.fetchall()

    # Ottimizza le istruzioni per ogni riga
    for url, istruzioni in righe:
        istruzioni_ottimizzate = ottimizza_istruzioni(istruzioni)

        # Aggiorna il database con le istruzioni ottimizzate
        cursor.execute("UPDATE Ricette SET Istruzioni = ? WHERE URL = ?", (istruzioni_ottimizzate, url))

    # Salva le modifiche e chiudi la connessione
    conn.commit()
    conn.close()

    print("Ottimizzazione completata.")

if __name__ == "__main__":
    main()

Ottimizzazione completata.


# 5. Partendo dalla lista ingredienti, il seguente codice calcola calorie, proteine e grassi utilizzando prima i dati salvati nel DB locale EXCEL e, solo qualora serva, le API dell'USDA FoodData Central. Fornisce sia i valori totali che parziali (per 100g). In aggiunta aggiorna il DB ogni volta che viene cercato un nuovo ingrediente in modo da eviteare di ripetere le API call.

In [6]:
!pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.2 MB/s eta 0:00:00


In [18]:
import requests
import sqlite3
import os
import re
import logging
from typing import Optional, Dict, Tuple
from rapidfuzz import process, fuzz
import pandas as pd

class NutritionDataManager:
    def __init__(self, db_name: str, api_key: str):
        self.db_name = db_name
        self.api_key = api_key
        self.ensure_table_exists()
        self.cache_db = self.load_cache()
        self.normalize_energy_kcal()  # Normalizza i dati dopo aver caricato la cache

    def ensure_table_exists(self):
        """
        Assicura che la tabella 'ingredienti' esista nel database,
        creandola se necessario con le colonne di base.
        """
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        c.execute('''
            CREATE TABLE IF NOT EXISTS ingredienti (
                Nome_ingrediente TEXT PRIMARY KEY,
                Descrizione_USDA TEXT,
                USDA_match BOOLEAN,
                Similarity_score REAL,
                cibi_non_trovati BOOLEAN,
                Energy_KCAL REAL,
                Energy_KJ REAL,
                Energy_Atwater_General_Factors__KCAL REAL
            )
        ''')
        conn.commit()
        conn.close()

    def load_cache(self) -> pd.DataFrame:
        """
        Carica i dati dalla tabella 'ingredienti' in un DataFrame pandas.
        """
        conn = sqlite3.connect(self.db_name)
        cache_db = pd.read_sql_query("SELECT * FROM ingredienti", conn)
        conn.close()
        return cache_db

    def normalize_energy_kcal(self):
        """
        Normalizza i dati nella tabella 'ingredienti', riempiendo i valori mancanti
        di 'Energy_KCAL' con i valori della colonna 'Energy_Atwater_General_Factors__KCAL'
        se presenti.
        """
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()

        # Aggiorna i valori mancanti di 'Energy_KCAL'
        update_query = '''
        UPDATE ingredienti
        SET Energy_KCAL = Energy_Atwater_General_Factors__KCAL
        WHERE Energy_KCAL IS NULL AND Energy_Atwater_General_Factors__KCAL IS NOT NULL
        '''
        c.execute(update_query)

        conn.commit()
        conn.close()

        # Aggiorna la cache locale
        self.cache_db = self.load_cache()
        logging.info("Normalizzazione dei valori Energy_KCAL completata.")

    @staticmethod
    def custom_scorer(query, choice, **kwargs):
        # Rimuove la virgola dalla scelta per il confronto
        choice_no_comma = choice.replace(',', '')

        # Calcola diversi punteggi
        ratio = fuzz.ratio(query.lower(), choice_no_comma.lower())
        partial_ratio = fuzz.partial_ratio(query.lower(), choice_no_comma.lower())
        token_sort_ratio = fuzz.token_sort_ratio(query.lower(), choice_no_comma.lower())

        # Da più peso alle corrispondenze all'inizio della stringa
        if choice_no_comma.lower().startswith(query.lower()):
            return max(ratio, partial_ratio, token_sort_ratio) + 50

        # Restituisce il punteggio massimo tra i diversi metodi
        return max(ratio, partial_ratio, token_sort_ratio)

    @staticmethod
    def get_best_match(ingredient, food_descriptions):
        best_match = process.extractOne(
            ingredient,
            food_descriptions,
            scorer=NutritionDataManager.custom_scorer,
            score_cutoff=80
        )
        return best_match if best_match else (None, 0, -1)

    def get_nutritional_data(self, ingredient: str) -> Tuple[Optional[Dict], Optional[str], bool, float]:
        """
        Recupera i dati nutrizionali per un ingrediente, prima cercando nella cache locale,
        poi facendo una chiamata API a USDA se necessario.
        """
        ingredient_lower = ingredient.lower()
        cache_match = self.cache_db[self.cache_db['Nome_ingrediente'].str.lower() == ingredient_lower]

        if not cache_match.empty:
            logging.info(f"Ingrediente '{ingredient}' trovato nella cache locale")
            return (
                cache_match.iloc[0].to_dict(),
                None,
                cache_match.iloc[0]['USDA_match'],
                cache_match.iloc[0]['Similarity_score']
            )

        search_url = f"https://api.nal.usda.gov/fdc/v1/foods/search?query={ingredient}&api_key={self.api_key}"
        try:
            response = requests.get(search_url)
            response.raise_for_status()
            data = response.json()
        except requests.RequestException as e:
            logging.error(f"Errore nella richiesta API per '{ingredient}': {str(e)}")
            return None, search_url, False, 0
        except ValueError as e:
            logging.error(f"Errore nel parsing della risposta JSON per '{ingredient}': {str(e)}")
            return None, search_url, False, 0

        if 'foods' in data and data['foods']:
            food_descriptions = [food['description'] for food in data['foods']]
            best_match = self.get_best_match(ingredient, food_descriptions)

            if best_match:
                match, score, _ = best_match  # Aggiunto _ per catturare il terzo valore
                fuzzy_match = next((food for food in data['foods'] if food['description'] == match), None)

                if fuzzy_match:
                    processed_data = self.process_food_data(fuzzy_match, ingredient)
                    self.add_to_cache(processed_data, ingredient, True, score)
                    return processed_data, search_url, True, score

        return self.handle_ingredient_not_found(ingredient, search_url)

    def handle_ingredient_not_found(self, ingredient: str, search_url: str) -> Tuple[Dict, str, bool, float]:
        """
        Gestisce il caso in cui un ingrediente non venga trovato nel database USDA.
        """
        logging.warning(f"Ingrediente '{ingredient}' non trovato nel DB USDA")
        not_found_data = {
            'Nome_ingrediente': ingredient,
            'Descrizione_USDA': None,
            'USDA_match': False,
            'Similarity_score': 0,
            'cibi_non_trovati': True
        }
        self.add_to_cache(not_found_data, ingredient, False, 0)
        return not_found_data, search_url, False, 0

    def process_food_data(self, food: Dict, original_name: str) -> Dict:
        """
        Elabora i dati grezzi dell'alimento restituiti dall'API USDA,
        strutturandoli nel formato richiesto per il database locale.
        """
        nutrients = food.get('foodNutrients', [])
        processed_data = {
            'Nome_ingrediente': original_name,
            'Descrizione_USDA': food.get('description', ''),
            'Energy_KCAL': None,
            'Energy_KJ': None
        }
        for nutrient in nutrients:
            nutrient_name = nutrient.get('nutrientName', '')
            nutrient_amount = nutrient.get('value')
            nutrient_unit = nutrient.get('unitName', '')

            if nutrient_amount is not None:
                if nutrient_name == 'Energy':
                    if nutrient_unit == 'KCAL':
                        processed_data['Energy_KCAL'] = nutrient_amount
                    elif nutrient_unit == 'kJ':
                        processed_data['Energy_KJ'] = nutrient_amount
                else:
                    # Creiamo il nome della colonna includendo l'unità di misura
                    nutrient_key = f"{re.sub(r'[^a-zA-Z0-9]+', '_', nutrient_name)}_{nutrient_unit}"
                    # Salviamo solo il valore numerico
                    processed_data[nutrient_key] = nutrient_amount

        return processed_data

    def add_to_cache(self, data: Dict, ingredient: str, usda_match: bool, similarity_score: float):
        """
        Aggiunge o aggiorna i dati nutrizionali di un ingrediente nel database locale,
        creando nuove colonne se necessario.
        """
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()

        existing_columns = [column[1] for column in c.execute("PRAGMA table_info(ingredienti)").fetchall()]
        for column in data.keys():
            if column not in existing_columns:
                c.execute(f"ALTER TABLE ingredienti ADD COLUMN {column} REAL")

        columns = ', '.join(data.keys())
        placeholders = ', '.join(['?' for _ in data])
        query = f"INSERT OR REPLACE INTO ingredienti ({columns}, USDA_match, Similarity_score, cibi_non_trovati) VALUES ({placeholders}, ?, ?, ?)"

        values = list(data.values()) + [usda_match, similarity_score, data.get('cibi_non_trovati', False)]
        c.execute(query, values)

        conn.commit()
        conn.close()

        self.cache_db = self.load_cache()

    def update_nutritional_values_in_db(self, food_name: str, nutritional_values: Dict):
        """
        Aggiorna i valori nutrizionali di un alimento nel database locale.
        """
        self.add_to_cache(nutritional_values, food_name, True, 100)


In [19]:
import json
import sqlite3
import logging
from typing import Dict, List
#from NutritionDataManager import NutritionDataManager

# Configurazione del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def get_recipes_from_db(db_name: str) -> List[Dict]:
    """
    Recupera tutte le ricette dal database.

    Args:
    db_name (str): Nome del file del database SQLite.

    Returns:
    List[Dict]: Lista di dizionari, ogni dizionario rappresenta una ricetta.
    """
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute("SELECT URL, Ingredients_JSON FROM ricette")  # Eseguiamo la query per ottenere URL e ingredienti
    recipes = [{"URL": row[0], "Ingredients": json.loads(row[1])} for row in cursor.fetchall()]  # Convertiamo il JSON in dizionario

    conn.close()
    return recipes

def process_ingredients(nutrition_manager: NutritionDataManager, recipes: List[Dict]):
    """
    Elabora gli ingredienti di tutte le ricette e li aggiunge al database degli ingredienti.

    Args:
    nutrition_manager (NutritionDataManager): Istanza del gestore dei dati nutrizionali.
    recipes (List[Dict]): Lista delle ricette con i loro ingredienti.
    """
    for recipe in recipes:
        logging.info(f"Elaborazione degli ingredienti per la ricetta: {recipe['URL']}")

        for ingredient, quantity in recipe['Ingredients'].items():
            # Otteniamo i dati nutrizionali per l'ingrediente
            nutritional_data, _, usda_match, similarity_score = nutrition_manager.get_nutritional_data(ingredient)

            if nutritional_data:
                if not usda_match:
                    # Registriamo gli ingredienti non trovati per una revisione manuale
                    logging.warning(f"Ingrediente non trovato nel database USDA: {ingredient}")
            else:
                logging.error(f"Impossibile ottenere dati nutrizionali per: {ingredient}")

def main():
    db_name = 'ricette.db'
    api_key = 'lRRUhBqakDfuwdjbdfEEck02bklmh6ml32q5ANxp'

    # Inizializziamo il gestore dei dati nutrizionali
    nutrition_manager = NutritionDataManager(db_name, api_key)

    # Recuperiamo le ricette dal database
    recipes = get_recipes_from_db(db_name)

    # Elaboriamo gli ingredienti di tutte le ricette
    process_ingredients(nutrition_manager, recipes)

    logging.info("Elaborazione degli ingredienti completata.")

if __name__ == "__main__":
    main()

ERROR:root:Errore nella richiesta API per ':': 500 Server Error: Internal Server Error for url: https://api.nal.usda.gov/fdc/v1/foods/search?query=:&api_key=lRRUhBqakDfuwdjbdfEEck02bklmh6ml32q5ANxp
ERROR:root:Impossibile ottenere dati nutrizionali per: :
ERROR:root:Errore nella richiesta API per 'butter ()': 500 Server Error: Internal Server Error for url: https://api.nal.usda.gov/fdc/v1/foods/search?query=butter%20()&api_key=lRRUhBqakDfuwdjbdfEEck02bklmh6ml32q5ANxp
ERROR:root:Impossibile ottenere dati nutrizionali per: butter ()
ERROR:root:Errore nella richiesta API per 'eggs ()': 500 Server Error: Internal Server Error for url: https://api.nal.usda.gov/fdc/v1/foods/search?query=eggs%20()&api_key=lRRUhBqakDfuwdjbdfEEck02bklmh6ml32q5ANxp
ERROR:root:Impossibile ottenere dati nutrizionali per: eggs ()
ERROR:root:Errore nella richiesta API per 'whole milk ()': 500 Server Error: Internal Server Error for url: https://api.nal.usda.gov/fdc/v1/foods/search?query=whole%20milk%20()&api_key=lRRU

In [ ]:
''' PER NORMALIZZARE LA TABELLA RICETTE A MANO'''

import sqlite3

# Connessione al database
conn = sqlite3.connect('ricette.db')
cursor = conn.cursor()

# Valore JSON da inserire
json_value = '{"flour 00": 250.0, "manitoba flour": 250.0, "warm milk": 250.0, "butter": 95.0, "sugar": 70.0, "medium eggs": 2.0, "fresh brewer\'s yeast": 12.0, "powdered sugar": 150.0, "dark chocolate": 50.0}'

# Comando SQL per aggiornare la colonna Ingredients_JSON
update_query = """
UPDATE ricette
SET Ingredients_JSON = ?
WHERE URL = ?;
"""

# Esecuzione del comando SQL con i parametri
cursor.execute(update_query, (json_value, 'URL ricetta')) # <------------ URL RICETTA QUI

# Commit delle modifiche
conn.commit()

# Chiusura della connessione
conn.close()

# Valori nutrizionali totali per ogni ricetta per porzione con salvataggio in DB

In [11]:
import sqlite3
import json

def calcola_quantita_grammi(ingrediente, quantita):
    try:
        quantita_float = float(quantita)

        if any(keyword in ingrediente.lower() for keyword in ["eggs", "uova"]):
            if quantita_float < 20:
                return quantita_float * 50  # 50g per uovo intero
            else:
                return quantita_float

        elif any(keyword in ingrediente.lower() for keyword in ["albumi", "albumen", "white egg", "egg white"]):
            if quantita_float < 20:
                return quantita_float * 30  # 30g per albume
            else:
                return quantita_float

        elif any(keyword in ingrediente.lower() for keyword in ["tuorli", "yolk"]):
            if quantita_float < 20:
                return quantita_float * 20  # 20g per tuorlo
            else:
                return quantita_float

        else:
            return quantita_float

    except ValueError:
        print(f"Errore nella conversione della quantità per {ingrediente}: {quantita}")
        return None

def rinomina_colonna_se_necessario(cur):
    cur.execute("PRAGMA table_info(ingredienti)")
    colonne_esistenti = [info[1] for info in cur.fetchall()]

    if 'Total_lipid_fat_G' not in colonne_esistenti and 'Total_lipid_fat__G' in colonne_esistenti:
        cur.execute("ALTER TABLE ingredienti RENAME COLUMN Total_lipid_fat__G TO Total_lipid_fat_G")
        print("Colonna rinominata da 'Total_lipid_fat__G' a 'Total_lipid_fat_G'.")

def calcola_valori_nutrizionali(url_ricetta, cur):
    cur.execute("SELECT URL, Ingredients_JSON, Dosi_per FROM ricette WHERE URL = ?", (url_ricetta,))
    risultato = cur.fetchone()

    if not risultato:
        print(f"Nessuna ricetta trovata per l'URL: {url_ricetta}")
        return None

    url_ricetta, ingredienti_json, dosi_per = risultato

    try:
        dosi_per = float(dosi_per)
        if dosi_per <= 0:
            raise ValueError("Dosi_per deve essere maggiore di zero")
    except (ValueError, TypeError) as e:
        print(f"Errore: {e} - Dosi_per non valido per la ricetta {url_ricetta}. Usando 1 come valore predefinito.")
        dosi_per = 1

    try:
        ingredienti = json.loads(ingredienti_json)
    except json.JSONDecodeError as e:
        print(f"Errore nel decodificare il JSON per la ricetta: {url_ricetta}. Errore: {e}")
        return None

    totali = {}

    colonne_da_escludere = ['Descrizione_USDA', 'USDA_match', 'Similarity_score', 'cibi_non_trovati', 'Nome_ingrediente']

    for ingrediente, quantita in ingredienti.items():
        quantita_grammi = calcola_quantita_grammi(ingrediente, quantita)

        if quantita_grammi is None:
            continue

        cur.execute("SELECT * FROM ingredienti WHERE Nome_ingrediente = ?", (ingrediente,))
        valori = cur.fetchone()

        if valori:
            colonne = [description[0] for description in cur.description]
            for col, val in zip(colonne, valori):
                if col not in colonne_da_escludere and val is not None:
                    try:
                        val_float = float(val)
                        valore_scalato = (val_float * quantita_grammi) / 100
                        totali[col] = totali.get(col, 0) + valore_scalato
                        # Aggiungi print per debug
                        if col == 'Total_lipid_fat_G':
                            print(f"Debug - Aggiungendo grassi per {ingrediente}: {val_float} * {quantita_grammi} / 100 = {valore_scalato}")
                    except ValueError:
                        print(f"Errore nella conversione del valore per {col}: {val}")
        else:
            print(f"Nessun dato nutrizionale trovato per l'ingrediente: {ingrediente}")

    print(f"Dividendo i valori nutrizionali per {dosi_per} porzioni")
    for nutriente in totali:
        totali[nutriente] = round(totali[nutriente] / dosi_per, 2)

    return totali

def aggiungi_colonne_se_non_esistono(cur):
    colonne_da_aggiungere = [
        "Calorie_totali_porzione", "Proteine_totali_porzione_G",
        "Grassi_totali_porzione_G", "Cholesterolo_totale_porzione_MG"
    ]

    cur.execute("PRAGMA table_info(ricette)")
    colonne_esistenti = [info[1] for info in cur.fetchall()]

    for colonna in colonne_da_aggiungere:
        if colonna not in colonne_esistenti:
            cur.execute(f"ALTER TABLE ricette ADD COLUMN {colonna} REAL")

# Connessione al database
conn = sqlite3.connect('ricette.db')
cur = conn.cursor()

try:
    # Rinomina la colonna se necessario
    rinomina_colonna_se_necessario(cur)

    # Aggiungi le nuove colonne se non esistono
    aggiungi_colonne_se_non_esistono(cur)

    # Ottieni tutte le URL delle ricette
    cur.execute("SELECT URL FROM ricette")
    url_ricette = cur.fetchall()

    ricette_aggiornate = 0

    for (url,) in url_ricette:
        valori_nutrizionali = calcola_valori_nutrizionali(url, cur)
        if valori_nutrizionali:
            update_query = """
            UPDATE ricette SET
                Calorie_totali_porzione = ?,
                Proteine_totali_porzione_G = ?,
                Grassi_totali_porzione_G = ?,
                Cholesterolo_totale_porzione_MG = ?
            WHERE URL = ?
            """
            valori_da_inserire = (
                valori_nutrizionali.get('Energy_KCAL', 0),
                valori_nutrizionali.get('Protein_G', 0),
                valori_nutrizionali.get('Total_lipid_fat_G', 0),  # Verifica del nome della chiave
                valori_nutrizionali.get('Cholesterol_MG', 0),
                url
            )
            cur.execute(update_query, valori_da_inserire)
            conn.commit()
            ricette_aggiornate += 1

            # Stampa i valori salvati nel database
            print(f"\nValori nutrizionali salvati per la ricetta {url}:")
            print(f"  Calorie totali per porzione: {valori_da_inserire[0]}")
            print(f"  Proteine totali per porzione (g): {valori_da_inserire[1]}")
            print(f"  Grassi totali per porzione (g): {valori_da_inserire[2]}")
            print(f"  Colesterolo totale per porzione (mg): {valori_da_inserire[3]}")

            # Print di debug
            print(f"Debug - Valori calcolati:")
            for key, value in valori_nutrizionali.items():
                print(f"  {key}: {value}")

finally:
    conn.close()

print(f"\nElaborazione completata con successo. {ricette_aggiornate} ricette sono state aggiornate con i valori nutrizionali per porzione.")


Colonna rinominata da 'Total_lipid_fat__G' a 'Total_lipid_fat_G'.
Debug - Aggiungendo grassi per 240 g: 30.2 * 0.0 / 100 = 0.0
Debug - Aggiungendo grassi per water: 0.0 * 180.0 / 100 = 0.0
Debug - Aggiungendo grassi per powdered sugar: 0.0 * 150.0 / 100 = 0.0
Debug - Aggiungendo grassi per eggs: 27.5 * 100.0 / 100 = 27.5
Debug - Aggiungendo grassi per honey: 0.0 * 20.0 / 100 = 0.0
Errore nella conversione della quantità per seed oil: to taste
Debug - Aggiungendo grassi per hazelnut spread cream: 30.0 * 60.0 / 100 = 18.0
Debug - Aggiungendo grassi per berry jam: 0.07 * 60.0 / 100 = 0.042
Dividendo i valori nutrizionali per 6.0 porzioni

Valori nutrizionali salvati per la ricetta https://ricette.giallozafferano.it/Banana-bread.html:
  Calorie totali per porzione: 298.03
  Proteine totali per porzione (g): 1.3
  Grassi totali per porzione (g): 7.59
  Colesterolo totale per porzione (mg): 2.0
Debug - Valori calcolati:
  Energy_KCAL: 298.03
  Protein_G: 1.3
  Total_lipid_fat_G: 7.59
  Total

# Algoritmo meal planner


In [ ]:
import sqlite3
import json
from collections import defaultdict

def get_db_connection():
    conn = sqlite3.connect('ricette.db')
    conn.row_factory = sqlite3.Row
    return conn

def get_random_recipe(conn, meal_type, max_time, used_recipes):
    query = """
    SELECT * FROM ricette
    WHERE Tempo_preparazione_totale <= ?
    AND Tipologia_piatti LIKE ?
    AND Titolo NOT IN ({})
    ORDER BY RANDOM() LIMIT 1
    """.format(",".join("?" * len(used_recipes)))

    meal_type_mapping = {
        'colazione': '%colazione%',
        'pranzo': '%primi%',
        'cena': '%secondi%'
    }

    db_meal_type = meal_type_mapping.get(meal_type, '%primi%')

    recipe = conn.execute(query, [max_time, db_meal_type] + list(used_recipes)).fetchone()
    return recipe

def generate_daily_meal_plan(conn, max_calories_per_day, max_total_time, min_protein, used_recipes):
    daily_plan = {'colazione': None, 'pranzo': None, 'cena': None}
    meal_types = ['colazione', 'pranzo', 'cena']

    best_plan = None
    best_protein = 0
    best_calories = 0

    max_attempts = 200  # Aumentiamo significativamente il numero di tentativi
    for attempt in range(max_attempts):
        total_calories = 0
        total_protein = 0
        total_fats = 0
        current_plan = {}

        for meal in meal_types:
            recipe = get_random_recipe(conn, meal, max_total_time, used_recipes)
            if recipe:
                current_plan[meal] = recipe
                total_calories += recipe['Calorie_totali_porzione'] or 0
                total_protein += float(recipe['Proteine_totali_porzione_G'] or 0)
                total_fats += float(recipe['Grassi_totali_porzione_G'] or 0)

        # Verifichiamo se questo piano è il migliore finora
        if total_protein > best_protein or (total_protein == best_protein and total_calories <= max_calories_per_day):
            best_plan = current_plan
            best_protein = total_protein
            best_calories = total_calories

        # Se troviamo un piano che soddisfa entrambi i criteri, lo restituiamo immediatamente
        if total_calories <= max_calories_per_day and total_protein >= min_protein:
            return current_plan, total_calories, total_protein, total_fats, False

    # Se non troviamo un piano perfetto, restituiamo il miglior piano trovato
    if best_plan:
        protein_difference = max(0, min_protein - best_protein)
        return best_plan, best_calories, best_protein, sum(float(recipe['Grassi_totali_porzione_G'] or 0) for recipe in best_plan.values()), protein_difference
    else:
        return None, 0, 0, 0, min_protein  # Caso in cui non troviamo alcun piano valido

def generate_weekly_meal_plan(max_calories_per_day, max_total_time, min_protein):
    conn = get_db_connection()
    weekly_plan = []
    used_recipes = set()
    total_protein_difference = 0

    for day in range(7):
        daily_plan, total_calories, total_protein, total_fats, protein_difference = generate_daily_meal_plan(conn, max_calories_per_day, max_total_time, min_protein, used_recipes)

        if daily_plan is None:
            print(f"Impossibile generare un piano valido per il giorno {day+1}. Interrompo la generazione del piano settimanale.")
            break

        # Aggiungi le ricette utilizzate a used_recipes
        for meal, recipe in daily_plan.items():
            if recipe:
                used_recipes.add(recipe['Titolo'])

        weekly_plan.append((daily_plan, total_calories, total_protein, total_fats))
        total_protein_difference += protein_difference

    conn.close()
    return weekly_plan, total_protein_difference

def generate_shopping_list(weekly_plan):
    shopping_list = defaultdict(float)

    for daily_plan, _, _, _ in weekly_plan:
        for meal, recipe in daily_plan.items():
            if recipe:
                ingredients = json.loads(recipe['Ingredienti_JSON'])
                for ingredient, quantity in ingredients.items():
                    try:
                        shopping_list[ingredient] += float(quantity)
                    except ValueError:
                        print(f"Quantità non numerica '{quantity}' per ingrediente '{ingredient}' ignorata.")

    return shopping_list

def create_todo_list(shopping_list):
    todo_list = [f"{quantity} g of {ingredient}" for ingredient, quantity in shopping_list.items()]
    return todo_list

# Test dell'algoritmo
max_calories_per_day = 2500
max_total_time = 120  # 120 minuti per singola ricetta
min_protein = 75  # 100 grammi totali giornalieri

weekly_plan, total_protein_difference = generate_weekly_meal_plan(max_calories_per_day, max_total_time, min_protein)

if total_protein_difference > 0:
    print(f"\nATTENZIONE: Il piano settimanale è stato generato, ma c'è una differenza di {total_protein_difference:.2f}g di proteine in meno rispetto al minimo richiesto.")

# Stampa il piano pasti settimanale
print("\nPiano pasti settimanale:")
for day, (daily_plan, total_calories, total_protein, total_fats) in enumerate(weekly_plan, start=1):
    print(f"\nGiorno {day}:")
    for meal, recipe in daily_plan.items():
        if recipe:
            print(f"\n{meal.capitalize()}:")
            print(f"Titolo: {recipe['Titolo']}")
            print(f"Calorie: {recipe['Calorie_totali_porzione']} Kcal")
            print(f"Proteine: {int(float(recipe['Proteine_totali_porzione_G'] or 0))} g")
            print(f"Grassi: {int(float(recipe['Grassi_totali_porzione_G'] or 0))} g")
            print(f"Tempo totale (preparazione + cottura): {recipe['Tempo_preparazione_totale']} minuti")
        else:
            print(f"\n{meal.capitalize()}: Nessuna ricetta trovata che soddisfi i criteri")

    print(f"\nTotali giornalieri:")
    print(f"Calorie totali (Kcal): {int(total_calories)}/{max_calories_per_day}")
    print(f"Proteine totali (g): {int(total_protein)}/{min_protein}")
    print(f"Grassi totali (g): {int(total_fats)}")

# Genera lista della spesa
shopping_list = generate_shopping_list(weekly_plan)
print("\nLista della spesa:")
for ingredient, quantity in shopping_list.items():
    print(f"{quantity} g di {ingredient}")

# Genera to-do list
todo_list = create_todo_list(shopping_list)
print("\nTo-do list:")
for item in todo_list:
    print(f"[ ] {item}")



ATTENZIONE: Il piano settimanale è stato generato, ma c'è una differenza di 38.00g di proteine in meno rispetto al minimo richiesto.

Piano pasti settimanale:

Giorno 1:

Colazione:
Titolo: Tortino di cioccolato con cuore fondente
Calorie: 436.45 Kcal
Proteine: 6 g
Grassi: 36 g
Tempo totale (preparazione + cottura): 35 minuti

Pranzo:
Titolo: Calamarata
Calorie: 528.7 Kcal
Proteine: 52 g
Grassi: 6 g
Tempo totale (preparazione + cottura): 70 minuti

Cena:
Titolo: Frittata di zucchine
Calorie: 455.12 Kcal
Proteine: 21 g
Grassi: 10 g
Tempo totale (preparazione + cottura): 30 minuti

Totali giornalieri:
Calorie totali (Kcal): 1420/2500
Proteine totali (g): 80/75
Grassi totali (g): 54

Giorno 2:

Colazione:
Titolo: Pasta frolla morbida
Calorie: 2733.5 Kcal
Proteine: 79 g
Grassi: 247 g
Tempo totale (preparazione + cottura): 10 minuti

Pranzo:
Titolo: Spaghetti alla Carbonara
Calorie: 455.85 Kcal
Proteine: 24 g
Grassi: 10 g
Tempo totale (preparazione + cottura): 25 minuti

Cena:
Titolo: Ling

# APP Streamlit

In [ ]:
!pip install streamlit pyngrok
!pip install streamlit flask flask-ngrok pyngrok==4.1.1
!pip install -q streamlit
!pip install pyngrok
!pip install streamlit
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 0.964s


In [ ]:
import streamlit as st
from pyngrok import ngrok
import sqlite3
import json
from collections import defaultdict

# Configura ngrok
!ngrok authtoken 2VFbhDnJuMQciAPeNTN0xgXUFBe_71YpQ3CS2m1oTgaWsD1yd  # Sostituisci con il tuo token ngrok

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile /content/app.py

import streamlit as st
import sqlite3
import json
from collections import defaultdict

#elenco funzioni per app
def get_db_connection():
    conn = sqlite3.connect('ricette.db')
    conn.row_factory = sqlite3.Row
    return conn

def get_random_recipe(conn, meal_type, max_time, used_recipes):
    query = """
    SELECT * FROM ricette
    WHERE Tempo_preparazione_totale <= ?
    AND Tipologia_piatti LIKE ?
    AND Titolo NOT IN ({})
    ORDER BY RANDOM() LIMIT 1
    """.format(",".join("?" * len(used_recipes)))

    meal_type_mapping = {
        'colazione': '%colazione%',
        'pranzo': '%primi%',
        'cena': '%secondi%'
    }

    db_meal_type = meal_type_mapping.get(meal_type, '%primi%')

    recipe = conn.execute(query, [max_time, db_meal_type] + list(used_recipes)).fetchone()
    return recipe

def generate_daily_meal_plan(conn, max_calories_per_day, max_total_time, min_protein, used_recipes):
    daily_plan = {'colazione': None, 'pranzo': None, 'cena': None}
    meal_types = ['colazione', 'pranzo', 'cena']

    best_plan = None
    best_protein = 0
    best_calories = 0

    max_attempts = 200  # Aumentiamo significativamente il numero di tentativi
    for attempt in range(max_attempts):
        total_calories = 0
        total_protein = 0
        total_fats = 0
        current_plan = {}

        for meal in meal_types:
            recipe = get_random_recipe(conn, meal, max_total_time, used_recipes)
            if recipe:
                current_plan[meal] = recipe
                total_calories += recipe['Calorie_totali_porzione'] or 0
                total_protein += float(recipe['Proteine_totali_porzione_G'] or 0)
                total_fats += float(recipe['Grassi_totali_porzione_G'] or 0)

        # Verifichiamo se questo piano è il migliore finora
        if total_protein > best_protein or (total_protein == best_protein and total_calories <= max_calories_per_day):
            best_plan = current_plan
            best_protein = total_protein
            best_calories = total_calories

        # Se troviamo un piano che soddisfa entrambi i criteri, lo restituiamo immediatamente
        if total_calories <= max_calories_per_day and total_protein >= min_protein:
            return current_plan, total_calories, total_protein, total_fats, False

    # Se non troviamo un piano perfetto, restituiamo il miglior piano trovato
    if best_plan:
        protein_difference = max(0, min_protein - best_protein)
        return best_plan, best_calories, best_protein, sum(float(recipe['Grassi_totali_porzione_G'] or 0) for recipe in best_plan.values()), protein_difference
    else:
        return None, 0, 0, 0, min_protein  # Caso in cui non troviamo alcun piano valido

def generate_weekly_meal_plan(max_calories_per_day, max_total_time, min_protein):
    conn = get_db_connection()
    weekly_plan = []
    used_recipes = set()
    total_protein_difference = 0

    for day in range(7):
        daily_plan, total_calories, total_protein, total_fats, protein_difference = generate_daily_meal_plan(conn, max_calories_per_day, max_total_time, min_protein, used_recipes)

        if daily_plan is None:
            print(f"Impossibile generare un piano valido per il giorno {day+1}. Interrompo la generazione del piano settimanale.")
            break

        # Aggiungi le ricette utilizzate a used_recipes
        for meal, recipe in daily_plan.items():
            if recipe:
                used_recipes.add(recipe['Titolo'])

        weekly_plan.append((daily_plan, total_calories, total_protein, total_fats))
        total_protein_difference += protein_difference

    conn.close()
    return weekly_plan, total_protein_difference

def generate_shopping_list(weekly_plan):
    shopping_list = defaultdict(float)

    for daily_plan, _, _, _ in weekly_plan:
        for meal, recipe in daily_plan.items():
            if recipe:
                ingredients = json.loads(recipe['Ingredienti_JSON'])
                for ingredient, quantity in ingredients.items():
                    try:
                        shopping_list[ingredient] += float(quantity)
                    except ValueError:
                        print(f"Quantità non numerica '{quantity}' per ingrediente '{ingredient}' ignorata.")

    return shopping_list

def create_todo_list(shopping_list):
    todo_list = [f"{quantity} g of {ingredient}" for ingredient, quantity in shopping_list.items()]
    return todo_list


st.title("Pianificatore Settimanale di Pasti")

# Input utente
max_calories_per_day = st.number_input("Calorie massime giornaliere", min_value=1000, max_value=5000, value=2500)
max_total_time = st.number_input("Tempo massimo di preparazione (minuti)", min_value=10, max_value=240, value=120)
min_protein = st.number_input("Proteine minime giornaliere (g)", min_value=20, max_value=200, value=100)


if st.button("Genera Piano Settimanale"):
    weekly_plan, total_protein_difference = generate_weekly_meal_plan(max_calories_per_day, max_total_time, min_protein)

    if total_protein_difference > 0:
        st.warning(f"ATTENZIONE: Il piano settimanale è stato generato, ma c'è una differenza di {total_protein_difference:.2f}g di proteine in meno rispetto al minimo richiesto.")

    # Visualizza il piano pasti settimanale in 7 colonne
    st.subheader("Piano pasti settimanale:")

    cols = st.columns(7)
    for day, (daily_plan, total_calories, total_protein, total_fats) in enumerate(weekly_plan, start=1):
        with cols[day-1]:
            st.markdown(f"**Giorno {day}**")
            for meal, recipe in daily_plan.items():
                if recipe:
                    st.markdown(f"*{meal.capitalize()}:*")
                    st.markdown(f"**{recipe['Titolo']}**")
                    st.markdown(f"Calorie: {recipe['Calorie_totali_porzione']} Kcal")
                    st.markdown(f"Proteine: {int(float(recipe['Proteine_totali_porzione_G'] or 0))} g")
                    st.markdown(f"Grassi: {int(float(recipe['Grassi_totali_porzione_G'] or 0))} g")
                    st.markdown(f"Tempo: {recipe['Tempo_preparazione_totale']} min")
                else:
                    st.markdown(f"*{meal.capitalize()}:* Nessuna ricetta")

            st.markdown("**Totali giornalieri:**")
            st.markdown(f"Calorie: {int(total_calories)}/{max_calories_per_day} Kcal")
            st.markdown(f"Proteine: {int(total_protein)}/{min_protein} g")
            st.markdown(f"Grassi: {int(total_fats)} g")

    # Genera e visualizza la lista della spesa
    shopping_list = generate_shopping_list(weekly_plan)
    st.subheader("Lista della spesa:")
    for ingredient, quantity in shopping_list.items():
        st.write(f"{quantity} g di {ingredient}")

    # Genera e visualizza la to-do list
    todo_list = create_todo_list(shopping_list)
    st.subheader("To-do list:")
    for item in todo_list:
        st.checkbox(item)

# Aggiungi un pulsante per rigenerare il piano
if st.button("Rigenera Piano"):
    st.experimental_rerun()

Overwriting /content/app.py


PROVA UNA DELLE SEGUENTI OPZIONI.

scrivi il valore dell esternal url quando richiesto. in questo caso 34.125.84.73 senza la porta. IN CASO DI PROBLEMI INPROVVISI REINSTALLA I MODELLI (!PIP INSTALL...

In [ ]:
!streamlit run /content/app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.218.153:8501

npx: installed 22 in 2.825s
your url is: https://chilly-eels-write.loca.lt
  Stopping...
^C


In [ ]:
import subprocess
import time

# Avvia l'app Streamlit in background
!nohup streamlit run app.py &

# Attendi che l'app Streamlit sia avviata
time.sleep(10)

# Crea un tunnel con localtunnel
tunnel_process = subprocess.Popen(["npx", "localtunnel", "--port", "8501"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Leggi l'output di localtunnel
for line in tunnel_process.stdout:
    print(line.strip())
    if line.startswith("your url is:"):
        print("\nApri questo URL nel tuo browser per accedere all'app Streamlit:")
        print(line.strip())
        break

# Mantieni il notebook in esecuzione
print("\nPremi Ctrl+C per terminare l'app quando hai finito.")
try:
    tunnel_process.wait()
except KeyboardInterrupt:
    print("\nTerminazione dell'app...")
finally:
    # Termina i processi
    !pkill -f "streamlit run app.py"
    tunnel_process.terminate()

nohup: appending output to 'nohup.out'
your url is: https://tame-news-divide.loca.lt

Apri questo URL nel tuo browser per accedere all'app Streamlit:
your url is: https://tame-news-divide.loca.lt

Premi Ctrl+C per terminare l'app quando hai finito.

Terminazione dell'app...
